In [2]:
import pandas as pd
import pickle as pkl
import statistics
#from basketball_reference_web_scraper import client
#from basketball_reference_web_scraper.data import OutputType
#import hockey_scraper
import datetime
import requests
from sportsipy.nfl.boxscore import Boxscore, BoxscorePlayer
import warnings
from bs4 import BeautifulSoup, Comment
import time
import statistics
#from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import re, os
import time
import random
import datetime as dt
from datetime import datetime, timedelta
from IPython.display import clear_output
from baseball_scraper import batting_stats_range, pitching_stats_range, baseball_reference
from sportsipy.mlb.boxscore import Boxscores
import numpy as np
import openpyxl
from openpyxl.styles import Border, Side
from openpyxl.styles import Alignment, PatternFill, Font
warnings.simplefilter("ignore")

waiver_participants = ["Arron", "Adam", "Ben", "Brian", "Emmett", "Jordan", "Josh", "Phil"]

In [3]:
# Links for Later
#current_rosters_path = ""

### Starter Function

In [4]:
# names = ["Noah", "Julia", "Jared", "Alex", "Josh", "Jordan", "Dalton", "Jesse", "Raphy"]
# day = 31
# month = 12

# annual_stats = {"NFL":{"players":pd.DataFrame(), "teams":pd.DataFrame()}, "NHL":{"players":pd.DataFrame(), "teams":pd.DataFrame()},
#                                          "NBA":{"players":pd.DataFrame(), "teams":pd.DataFrame()}, "MLB":{"players":pd.DataFrame(), "teams":pd.DataFrame()}}

# league_stats_by_participant = {}
# for name in names:
#     league_stats_by_participant[name] = {"NFL":{"players":pd.DataFrame(), "teams":pd.DataFrame()}, "NHL":{"players":pd.DataFrame(), "teams":pd.DataFrame()},
#                                          "NBA":{"players":pd.DataFrame(), "teams":pd.DataFrame()}, "MLB":{"players":pd.DataFrame(), "teams":pd.DataFrame()}}

# pkl.dump(league_stats_by_participant.copy(), open("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/League Stats By Participant/League Stats By Participant {}-{}.pkl".format(str(month).zfill(2), str(day).zfill(2)), "wb"))
# pkl.dump(annual_stats.copy(), open("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Annual League Stats/Annual Stats {}-{}.pkl".format(str(month).zfill(2), str(day).zfill(2)), "wb"))

# General Imports and Functions

In [5]:
# import important files
nhl_name_conversions = pd.read_excel("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Team Name Conversions.xlsx",
                                     sheet_name="NHL Name Conversions", header = 2).iloc[:, 3:]

nfl_name_conversions = pd.read_excel("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Team Name Conversions.xlsx",
                                     sheet_name="NFL Name Conversions", header = 2).iloc[:, 1:]

mlb_name_conversions = pd.read_excel("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Team Name Conversions.xlsx",
                                     sheet_name="MLB Name Conversions", header = 2).iloc[:, 3:]

nba_name_conversions = pd.read_excel("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Team Name Conversions.xlsx",
                                     sheet_name="NBA Name Conversions", header = 2).iloc[:, 3:]

In [6]:
mlb_positions = ["Pitcher", "Catcher", "First Base", "Second Base", "Third Base", "Short Stop", "Left Field", "Center Field", "Right Field"]
nfl_positons = ["Quarterback", "Running Back", "Wide Reciever", "Tight End", "Punter", "Kicker", "Returner", "Defense"]
nhl_positions = ["Goalie", "Right Wing", "Centerman", "Left Wing", "Defenseman"]
nba_positions = ["Point Guard", "Shooting Guard", "Small Forward", "Power Forward", "Center"]

In [7]:
xlsx_columns = {1:"A", 2:"B", 3:"C", 4:"D", 5:"E", 6:"F", 7:"G", 8:"H", 9:"I", 10:"J", 11:"K", 12:"L", 13:"M", 14:"N", 15:"O", 16:"P", 17:"Q", 18:"R", 19:"S", 20:"T", 21:"U",
                22:"V", 23:"W", 24:"X", 25:"Y", 26:"Z", 27:"AA", 28:"AB", 29:"AC", 30:"AD", 31:"AE", 32:"AF", 33:"AG", 34:"AH", 35:"AI", 36:"AJ", 37:"AK", 38:"AL", 39:"AM", 40:"AN", 
                41:"AO", 42:"AP", 43:"AQ", 44:"AR", 45:"AS", 46:"AT", 47:"AU", 48:"AV", 49:"AW", 50:"AX", 51:"AY", 52:"AZ"}

In [8]:
def findTables(url):
    """Pulls all the relevant table ids from sports reference"""
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    divs = soup.findAll('div', id = "content")
    divs = divs[0].findAll("div", id=re.compile("^all"))
    ids = []
    for div in divs:
        searchme = str(div.findAll("table"))
        x = searchme[searchme.find("id=") + 3: searchme.find(">")]
        x = x.replace("\"", "")
        if len(x) > 0:
            ids.append(x)
    return(ids)

In [9]:
def find_nhl_season(day, month, year):
    """The url for scraping NHL games channges depending on which season were in, but the season doesn't directly correlate to the year. This returns the correct year to use in the URL based on the date
       If we play another season in 2023, make sure to update with further dates"""
    
    
    date = str(year) + "-" + str(month).zfill(2) + "-" + str(day).zfill(2)
    if date > "2022-06-30":
        return "2023"
    elif date > "2021-07-07":
        return "2022"
    else:
        return "2021"

In [10]:
important_dates = pd.read_excel("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/2023 Important Dates.xlsx", header = 1)
#start_dates = important_dates["Start Date"].dropna().apply(lambda x: x.strftime("%Y/%m/%d"))
#end_dates = important_dates["End Date"].dropna().apply(lambda x: x.strftime("%Y/%m/%d"))
playing_dates = important_dates["Playing Days"].apply(lambda x: x.strftime("%Y/%m/%d"))

In [11]:
# Get what period we are currently in for later use
start_dates = pd.date_range("1/3/2022", "12/20/2022", freq="14D")
start_dates = [x.strftime("%Y/%m/%d") for x in start_dates]

end_dates = pd.date_range("1/15/2023", "12/31/2023", freq="14D")
end_dates = [x.strftime("%Y/%m/%d") for x in end_dates]

dates = {num:{"start":[], "end":[]} for num in range(1,27)}
for num in dates:
    dates[num]["start"] = start_dates[num-1]
    dates[num]["end"] = end_dates[num-1]

def get_current_period(start_dates, year, month, day):
     
    if dt.datetime(year, month, day).strftime("%Y/%m/%d") in start_dates:
        trigger = 1
    else:
        trigger = 0    
    
    start_date = start_dates.copy()
    start_date.append(dt.datetime(year, month, day).strftime("%Y/%m/%d"))
    full_date_series = pd.Series(start_date).sort_values(ascending = True).reset_index(drop = True)
    current_period = full_date_series[full_date_series == dt.datetime(year, month, day).strftime("%Y/%m/%d")].index[0]
    
    return current_period + trigger

#### Scoring Dict

In [12]:
scoring_dict = {"NFL":{"All Purpose Yards":0.06, "All Purpose TDs":4.6, "Attempts":0.4, "Return Yards":0.15, "Kicking Yards":0.12, "Sacks/Turnovers":2.75},
                "NHL":{"Hockey Points":1, "Hockey Hits":0.6, "Hockey Blocks":0.65, "Penalty Minutes":-1.25, "Shots":0.22, "Goals Against":-1.1, "Saves":0.25},
                "NBA":{"Three Pointers":0.28, "Free Throws Made":0.14, "Rebounds":0.06, "Assists":0.11, "Steals":0.43, "Blocks":0.56, "Turnovers":-0.2, "Points":0.03},
                "MLB":{"Runs":0.56, "Home Runs":1.2, "RBIs":0.57, "Stolen Bases":2.5, "Walks":0.76, "Strikeouts":-0.25, "Total Bases":0.18, "Innings":1.1, "Earned Runs":-3.5, "Pitching Strikeouts":0.9, "Saves":0.75}}

positional_scoring_dict = {"NFL":{"Quarterback":["All Purpose Yards", "All Purpose TDs", "Attempts"],
                                  "Running Back":["All Purpose Yards", "All Purpose TDs", "Attempts"],
                                  "Wide Reciever":["All Purpose Yards", "All Purpose TDs", "Attempts"],
                                  "Tight End":["All Purpose Yards", "All Purpose TDs", "Attempts"],
                                  "Kicker":["Kicking Yards"],
                                  "Punter":["Kicking Yards"],
                                  "Returner":["Return Yards"],
                                  "Defense":["Sacks/Turnovers"]
                                  },
                            
                            "MLB":{"Catcher":["Runs", "Home Runs", "RBIs", "Stolen Bases", "Walks", "Strikeouts", "Total Bases"],
                                   "First Base":["Runs", "Home Runs", "RBIs", "Stolen Bases", "Walks", "Strikeouts", "Total Bases"],
                                   "Second Base":["Runs", "Home Runs", "RBIs", "Stolen Bases", "Walks", "Strikeouts", "Total Bases"],
                                   "Third Base":["Runs", "Home Runs", "RBIs", "Stolen Bases", "Walks", "Strikeouts", "Total Bases"],
                                   "Short Stop":["Runs", "Home Runs", "RBIs", "Stolen Bases", "Walks", "Strikeouts", "Total Bases"],
                                   "Outfield":["Runs", "Home Runs", "RBIs", "Stolen Bases", "Walks", "Strikeouts", "Total Bases"],
                                   "Pitcher":["Innings", "Earned Runs", "Pitching Strikeouts", "Saves"]
                                  },
                            
                            "NBA":{"Point Guard":["Three Pointers", "Free Throws Made", "Rebounds", "Assists", "Steals", "Blocks", "Turnovers", "Points"],
                                   "Shooting Guard":["Three Pointers", "Free Throws Made", "Rebounds", "Assists", "Steals", "Blocks", "Turnovers", "Points"],
                                   "Small Forward":["Three Pointers", "Free Throws Made", "Rebounds", "Assists", "Steals", "Blocks", "Turnovers", "Points"],
                                   "Power Forward":["Three Pointers", "Free Throws Made", "Rebounds", "Assists", "Steals", "Blocks", "Turnovers", "Points"],
                                   "Center":["Three Pointers", "Free Throws Made", "Rebounds", "Assists", "Steals", "Blocks", "Turnovers", "Points"]
                                  },
                            
                            "NHL":{"Centerman":["Hockey Points", "Hockey Hits", "Hockey Blocks", "Penalty Minutes", "Shots"],
                                   "Right Wing":["Hockey Points", "Hockey Hits", "Hockey Blocks", "Penalty Minutes", "Shots"],
                                   "Centerman":["Hockey Points", "Hockey Hits", "Hockey Blocks", "Penalty Minutes", "Shots"],
                                   "Left Wing":["Hockey Points", "Hockey Hits", "Hockey Blocks", "Penalty Minutes", "Shots"],
                                   "Defenseman":["Hockey Points", "Hockey Hits", "Hockey Blocks", "Penalty Minutes", "Shots"],
                                   "Goalie":["Saves", "Goals Against"]}
                           }

# Gathering Daily Stats

## NBA Statistics Collection

In [13]:
def get_daily_nba_stats(day, month, year):
    """A function that will return a cleaned pandas dataframe of daily NBA statistics for BOTH players and teams that played on the 
       given day.
       NOTE: All of day, month and year must be supplied as ints! 
       NOTE: This function can only be used for singular days, or the matching of location and opponent at the end of team data collection will return multiple/differing results, not singular locations and opponents!
       Returns: A dictionary with three keys of 'Players', 'Goalies', and 'Teams'"""
    
    # if "{}/{}/{}".format(year, month, day) not in playing dates:
    #     return pd.DataFrame()
    
    # We will eventually return this df
    final_stats = pd.DataFrame()
    final_team_df = pd.DataFrame()

    # We will continue taking dates in intigers for consistiency. However, when scraping directly from Hockey Reference, dates will be supplied back as strings. 
    # For slicing purposes, we build our date into a correctly formatted string.
    given_date = str(year).zfill(2) + "-" + str(month).zfill(2) + "-" + str(day).zfill(2)
    url_date = str(year).zfill(2) + "-" + str(month).zfill(2) + "-" + str(day).zfill(2)
    
    # Now using pandas read_html, scrape  the title info for all nhl games during the season, and slice just the ones for the given date
    season = str(int(find_nhl_season(day, month, year))) #WRITE A NEW FUNCTION THAT USES FINAL NBA SEASON DATES
    all_nbl_games_url = "https://www.basketball-reference.com/leagues/NBA_{}_games.html".format(season)
    webpage = requests.get(all_nbl_games_url)
    soup = BeautifulSoup(webpage.content, "html.parser")
    months = [x.text.lower() for x in soup.find_all("div", {"class":"filter"})[0].find_all('a')]
    all_nba_games = pd.DataFrame()
    for month in months:
        month_url = "https://www.basketball-reference.com/leagues/NBA_{}_games-{}.html".format(season, month)
        monthly_games_df = pd.read_html(month_url)[0]
        all_nba_games = all_nba_games.append(monthly_games_df)
       
    all_nba_games.Date = all_nba_games.Date.apply(lambda x: datetime.strptime(x, "%a, %b %d, %Y").strftime("%Y-%m-%d"))
    relevant_nba_games = all_nba_games[all_nba_games.Date == given_date]
  
    
    # If there were no games played on the given day, the df will be empty, and throw an error if we don't break before the next section
    if len(relevant_nba_games) <= 0:
        return pd.DataFrame()
    else:
        # Next, we build the url for the specific hockey reference url using data from each game in relevant_nhl_games
        for game in range(len(relevant_nba_games)):
            
            eventual_away_table_num = 9 if relevant_nba_games["Unnamed: 7"].iloc[game] == "OT" else 8
            
            home_team = relevant_nba_games["Home/Neutral"].iloc[game]
            home_abbr = nba_name_conversions[nba_name_conversions["Team Name"] == home_team]["Team Abbreviation"].iloc[0]
            away_team = relevant_nba_games["Visitor/Neutral"].iloc[game]
            away_abbr = nba_name_conversions[nba_name_conversions["Team Name"] == away_team]["Team Abbreviation"].iloc[0]
            
            url_insert = "".join(url_date.split("-")) + "0" + home_abbr
            game_url = "https://www.basketball-reference.com/boxscores/" + url_insert + ".html"
    
            time.sleep(5)
            game_tables = pd.read_html(game_url, header=1)
            
            away_stats = game_tables[0]
            away_stats = away_stats[(away_stats.Starters != "Reserves") & (pd.isna(away_stats.MP) == False)]
            away_totals = away_stats[away_stats.Starters == "Team Totals"]
            away_stats = away_stats[away_stats.Starters != "Team Totals"]
            away_stats["team"] = away_abbr
            
            home_stats = game_tables[eventual_away_table_num]
            home_stats = home_stats[(home_stats.Starters != "Reserves") & (pd.isna(home_stats.MP) == False)]
            home_totals = home_stats[home_stats.Starters == "Team Totals"]
            home_stats = home_stats[home_stats.Starters != "Team Totals"]
            home_stats["team"] = home_abbr
            
            
            home_totals["team"] = home_abbr
            away_totals["opponent"] = home_abbr
            
            home_totals["opponent"] = away_abbr
            away_totals["team"] = away_abbr
            
            home_points = home_totals.PTS.iloc[0]
            away_points = away_totals.PTS.iloc[0]
        
            home_totals["PTS"] = home_points
            away_totals["PTS"] = away_points
            
            home_totals["is_win"] = 1 if home_points > away_points else 0
            away_totals["is_win"] = 1 if away_points > home_points else 0

            webpage = requests.get(game_url)
            soup = BeautifulSoup(webpage.content, "html.parser")

            
            home_ids = [z["data-append-csv"] for z in [y for y in [x for x in soup.find_all("table") if "csk" in str(x)][eventual_away_table_num].find_all("th") if "csk" in str(y)]]
            away_ids = [z["data-append-csv"] for z in [y for y in [x for x in soup.find_all("table") if "csk" in str(x)][0].find_all("th") if "csk" in str(y)]]
           
            home_stats["id"] = home_ids
            home_stats = home_stats[(home_stats.MP != "Did Not Dress") & (home_stats.MP != "Not With Team") & (home_stats.MP != "Did Not Play") & (home_stats.MP != "Player Suspended") & (pd.isna(home_stats.MP) == False)]
            
            away_stats["id"] = away_ids
            away_stats = away_stats[(away_stats.MP != "Did Not Dress") & (away_stats.MP != "Not With Team") & (away_stats.MP != "Did Not Play") & (away_stats.MP != "Player Suspended") & (pd.isna(away_stats.MP) == False)]
            
            stats_addition = home_stats.append(away_stats)
            final_stats = final_stats.append(stats_addition).reset_index(drop=True)
            
            team_totals = home_totals.append(away_totals)
            final_team_df = final_team_df.append(team_totals).reset_index(drop = True)
            
    final_stats.MP = final_stats.MP.apply(lambda x: float(x.split(":")[0]) + float(x.split(":")[1])/60)
    #final_stats.PTS = final_stats.MP.apply(lambda x: int(x))
    
    final_team_df.PTS = final_team_df.PTS.apply(lambda x: int(x))
    #final_team_df.MP = final_team_df.MP.apply(lambda x: float(x["players"]["MP"].iloc[0].split(":")[0]) + float(x["players"]["MP"].iloc[0].split(":")[1])/60)
    
    return {"players":final_stats, "teams":final_team_df, "date":given_date}

## NHL Stats Collection

In [14]:
def get_daily_nhl_stats(day, month, year):
    """A function that will return a cleaned pandas dataframe of daily NHL statistics for BOTH players and teams that played on the 
       given day.
       NOTE: All of day, month and year must be supplied as ints! 
       NOTE: This function can only be used for singular days, or the matching of location and opponent at the end of team data collection will return multiple/differing results, not singular locations and opponents!
       Returns: A dictionary with three keys of 'Players', 'Goalies', and 'Teams'"""
    
    def pull_hits_and_blocks(player, stat, location):
        if location == "home":
            try:
                ans = home_advanced_stats[home_advanced_stats.Player == player][stat].iloc[0]
            except:
                return 0
        else:
            try: 
                ans = away_advanced_stats[away_advanced_stats.Player == player][stat].iloc[0]
            except:
                return 0
            
        return ans

                
    # if "{}/{}/{}".format(year, month, day) not in playing dates:
    #     return pd.DataFrame()
    
    # We will eventually return this df
    final_skater_df = pd.DataFrame()
    final_goalie_df = pd.DataFrame()
    final_team_df = pd.DataFrame()
    final_penalty_df = pd.DataFrame()

    # We will continue taking dates in intigers for consistiency. However, when scraping directly from Hockey Reference, dates will be supplied back as strings. 
    # For slicing purposes, we build our date into a correctly formatted string.
    given_date = str(year).zfill(2) + "-" + str(month).zfill(2) + "-" + str(day).zfill(2)
    
    # Now using pandas read_html, scrape  the title info for all nhl games during the season, and slice just the ones for the given date
    season = find_nhl_season(day, month, year)
    all_nhl_games_url = "https://www.hockey-reference.com/leagues/NHL_{}_games.html".format(season)
    all_nhl_games = pd.read_html(all_nhl_games_url)[0]
    try:
        playoff_games = pd.read_html(all_nhl_games_url)[1]
        all_nhl_games = all_nhl_games.append(playoff_games)
    except:
        None
    
    all_nhl_games = all_nhl_games.rename(columns = {"Date":"date", "Visitor":"visitor", "G":"away_goals",
                                                                            "Home":"home", "G.1":"home_goals", "Unnamed: 5":"is_ot",
                                                                         "Att.":"attendance", "LOG":"game_length", "Notes":"notes"})
    
    relevant_nhl_games = all_nhl_games[all_nhl_games.date == given_date]

    # If there were no games played on the given day, the df will be empty, and throw an error if we don't break before the next section
    if len(relevant_nhl_games) <= 0:
        return pd.DataFrame()
    
    else:
        
        # Next, we build the url for the specific hockey reference url using data from each game in relevant_nhl_games
        for game in range(len(relevant_nhl_games)):
            time.sleep(6)
            home_team = relevant_nhl_games.home.iloc[game]
            home_abbr = nhl_name_conversions[nhl_name_conversions["Team Name"] == home_team]["Team Abbreviation"].iloc[0]
            away_team = relevant_nhl_games.visitor.iloc[game]
            away_abbr = nhl_name_conversions[nhl_name_conversions["Team Name"] == away_team]["Team Abbreviation"].iloc[0]
            
            url_insert = "".join(given_date.split("-")) + "0" + home_abbr
            game_url = "https://www.hockey-reference.com/boxscores/" + url_insert + ".html"
            
            
            # Using the built url, scrape Hockey Reference and grab the tables (we know the order of them) for home and away
            # Stats for both skaters and goalies
            game_tables = pd.read_html(game_url, header=1)
        
            home_skater_stats = game_tables[4].drop(columns = "Rk")
            home_skater_stats["team"] = home_abbr
            
            home_goalie_stats = game_tables[5].drop(columns = "Rk")
            home_goalie_stats["team"] = home_abbr
            
            away_skater_stats = game_tables[2].drop(columns = "Rk")
            away_skater_stats["team"] = away_abbr
            
            away_goalie_stats = game_tables[3].drop(columns = "Rk")
            away_goalie_stats["team"] = away_abbr
            
            # Get and insert Pro Baseball Reference IDs into the stats dfs
            webpage = requests.get(game_url)
            soup = BeautifulSoup(webpage.content, "html.parser")
            
            
            away_names = [j.split(",")[1] + " " + j.split(",")[0] for j in [str(z).split("csk")[2].split('" ')[0].split("=")[1].strip('"') for z in [y for y in [x for x in soup.find_all("table") if "csk" in str(x)][0].find_all("tr") if "csk" in str(y)]]]
            away_ids = [str(z).split("data-append-csv")[1].split(" ")[0].split("=")[1].strip('"') for z in [y for y in [x for x in soup.find_all("table") if "csk" in str(x)][0].find_all("tr") if "csk" in str(y)]]
            home_names = [j.split(",")[1] + " " + j.split(",")[0] for j in [str(z).split("csk")[2].split('" ')[0].split("=")[1].strip('"') for z in [y for y in [x for x in soup.find_all("table") if "csk" in str(x)][2].find_all("tr") if "csk" in str(y)]]]
            home_ids = [str(z).split("data-append-csv")[1].split(" ")[0].split("=")[1].strip('"') for z in [y for y in [x for x in soup.find_all("table") if "csk" in str(x)][2].find_all("tr") if "csk" in str(y)]]
            
            away_id_dict = {away_names[n]:away_ids[n] for n in range(len(away_names))}
            home_id_dict = {home_names[n]:home_ids[n] for n in range(len(home_names))}
            
            home_skater_stats["player_id"] = home_skater_stats.Player.apply(lambda x: home_id_dict[x] if x not in ["TOTAL", "Empty Net"] else "")
            home_goalie_stats["player_id"] = home_goalie_stats.Player.apply(lambda x: home_id_dict[x] if x not in ["TOTAL", "Empty Net"] else "")
            #home_goalie_stats["goals_against"] = home_goalie_stats.SA - home_goalie_stats.SV
            
            away_skater_stats["player_id"] = away_skater_stats.Player.apply(lambda x: away_id_dict[x] if x not in ["TOTAL", "Empty Net"] else "")
            away_goalie_stats["player_id"] = away_goalie_stats.Player.apply(lambda x: away_id_dict[x] if x not in ["TOTAL", "Empty Net"] else "")
            #away_goalie_stats["goals_against"] = away_goalie_stats.SA - away_goalie_stats.SV
            
            
            # Add home and away goals scored to later determine which team won
            home_skater_stats["team_goals_scored"] = home_skater_stats[home_skater_stats.Player == "TOTAL"].G.sum()
            away_skater_stats["team_goals_scored"] = away_skater_stats[away_skater_stats.Player == "TOTAL"].G.sum()
            
            home_skater_stats["team_goals_allowed"] = away_skater_stats[away_skater_stats.Player == "TOTAL"].G.sum()
            away_skater_stats["team_goals_allowed"] = home_skater_stats[home_skater_stats.Player == "TOTAL"].G.sum()
            
            
            # Grab the advanced stats table to later get hits and blocks
            advanced_game_tables = pd.read_html(game_url)
            
            home_advanced_stats = advanced_game_tables[13]
            home_advanced_stats["team"] = home_abbr
            
            
            away_advanced_stats = advanced_game_tables[6]
            away_advanced_stats["team"] = away_abbr
            
            # Pull hits and blocks data from a seperate table grabbed above
            home_skater_stats["hits"] = home_skater_stats.apply(lambda x: pull_hits_and_blocks(x.Player, "HIT", "home"), axis = 1)
            home_skater_stats["blocks"] = home_skater_stats.apply(lambda x: pull_hits_and_blocks(x.Player, "BLK", "home"), axis = 1)
            
            
            away_skater_stats["hits"] = away_skater_stats.apply(lambda x: pull_hits_and_blocks(x.Player, "HIT", "away"), axis = 1)
            away_skater_stats["blocks"] = away_skater_stats.apply(lambda x: pull_hits_and_blocks(x.Player, "BLK", "away"), axis = 1)
            
            
            
            # Now that we have the stats from the game for both home and away skaters and goalies, we can add them game by game (in the loop)
            # to our final tables and rename them as needed
            skater_addition = pd.concat([home_skater_stats, away_skater_stats])
            final_skater_df = final_skater_df.append(skater_addition).reset_index(drop=True)
            
            goalie_addition = pd.concat([home_goalie_stats, away_goalie_stats])
            final_goalie_df = final_goalie_df.append(goalie_addition)
            
            # Now grab penalty data
            penalty_table = pd.read_html(game_url)[1]
            penalty_table.columns = ["time", "team", "player","penalty","penalty_minutes"]
            
            
#             .rename(columns = {"1st Period": "time", "1st Period.1": "team", "1st Period.2":"player",
#                                                                         "1st Period.3": "penalty", "1st Period.4":"penalty_minutes"})
            
            penalty_table = penalty_table[penalty_table.player.str.contains("Period") == False].drop(columns=["time"])
            final_penalty_df = final_penalty_df.append(penalty_table)
            
    
            
               
    
    # Finally, after adding together all the night's games' stats, we can make final visual edits to the tables, and add columns for some extra stats
    final_skater_df = final_skater_df.rename(columns = {"Player":"player", "G":"goals", "A": "assists", "PTS":"points", "+/-":"plus_minus",
                                                        "PIM": "penalty_minutes", "EV":"even_strength_goals", "PP":"power_play_goals",
                                                        "SH":"short_handed_goals", "GW":"game_winning_goals", "EV.1":"even_strength_assists",
                                                         "PP.1":"power_play_assists", "SH.1": "short_handed_assists", "S":"shots", "TOI":"time_on_ice"})
    final_skater_df["power_play_points"] = final_skater_df.power_play_goals + final_skater_df.power_play_assists
    final_skater_df["short_handed_points"] = final_skater_df.short_handed_goals + final_skater_df.short_handed_assists
    final_skater_df["time_on_ice"] = final_skater_df["time_on_ice"].apply(lambda x: float(x.split(":")[0]) + float(x.split(":")[1])/60 if pd.isna(x) == False else 0)
    final_skater_df = final_skater_df[['player', 'goals', 'assists', 'points', "power_play_points", "short_handed_points"] + [x for x in final_skater_df if x not in ['player', 'goals', 'assists', 'points', "power_play_points", "short_handed_points"]]]
    
    final_goalie_df = final_goalie_df.rename(columns = {"Player":"player", "DEC":"decision", "GA":"goals_against", "SA":"shots_against",
                                                        "SV":"saves", "SV%":"save_percentage", "SO":"shutouts", "PIM":"penalty_minutes",
                                                        "TOI":"time_on_ice"})
    final_goalie_df["time_on_ice"] = final_goalie_df["time_on_ice"].apply(lambda x: float(x.split(":")[0]) + float(x.split(":")[1])/60 if pd.isna(x) == False else 0)
    
    # As one last check, the scraping from Hockey Reference relies on the tables being in the right order. If they ever switch, the rename wont catch
    # this, it will fail, and we know to grab a different numbered table from the website for home, away skater stats
    correct_table_check = final_skater_df["power_play_goals"]
    
    
    # Now that we have skater and goalie stats for the home and away team finalized, grab team stats from rows titled "TOTAL"
    team_stats = final_skater_df[final_skater_df.player == "TOTAL"].dropna(axis=1).reset_index(drop=True).drop(columns = ["player", "time_on_ice"])
    team_stats = team_stats[["team"] + [x for x in team_stats if x not in ["team"]]]
    
    final_skater_df = final_skater_df[final_skater_df.player != "TOTAL"]
    final_goalie_df = final_goalie_df[final_goalie_df.player != "TOTAL"]
    
    
    
    return {"Skaters":final_skater_df, "Goalies":final_goalie_df, "Teams":team_stats, "Penalties":final_penalty_df, "date":given_date}

## NFL Stats Collection

In [15]:
def get_daily_nfl_stats(day, month, year):
    """A function that will return a cleaned pandas dataframe of daily NFL statistics for BOTH players and teams that played on the 
       given day.
       NOTE: All of day, month and year must be supplied as ints! 
       NOTE: This function can only be used for singular days, or the matching of location and opponent at the end of team data collection will return multiple/differing results, not singular locations and opponents!
       """ 
    
    # if "{}/{}/{}".format(year, month, day) not in playing dates:
    #     return pd.DataFrame()
    
    # We will eventually return this df
    final_offensive_df = pd.DataFrame()
    final_kicking_df = pd.DataFrame()
    final_defensive_df = pd.DataFrame()
    final_team_df = pd.DataFrame()
    final_snap_counts_df = pd.DataFrame()
    final_field_goal_df = pd.DataFrame()
    final_returns_df = pd.DataFrame()
    final_penalty_df = pd.DataFrame()
    

    # We will continue taking dates in intigers for consistiency. However, when scraping directly from Football Reference, dates will be supplied back as strings. 
    # For slicing purposes, we build our date into a correctly formatted string.
    given_date = str(year).zfill(2) + "-" + str(month).zfill(2) + "-" + str(day).zfill(2)
    
    # Now using pandas read_html, scrape  the title info for all nfl games during the season, and slice just the ones for the given date
    all_nfl_games_url = "https://www.pro-football-reference.com/years/2022/games.htm"
    all_nfl_games = pd.read_html(all_nfl_games_url)[0].rename(columns = {"Date":"date", "Week":"week", "Day":"day", "Time":"time", "Winner/tie":"winner/tie", "Loser/tie":"loser/tie", "PtsW":"winning_team_points",
                                                                     "PtsL":"losing_team_points", "YdsW":"winning_team_yards", "TOW":"winning_team_turnovers", "YdsL":"losing_team_yards", "TOL":"losing_team_turnovers"})
    relevant_nfl_games = all_nfl_games[all_nfl_games.date == given_date]
    
    # If there were no games played on the given day, the df will be empty, and throw an error if we don't break before the next section
    if len(relevant_nfl_games) <= 0:
        return pd.DataFrame()
    
    else:
        
        # Next, we build the url for the specific hockey reference url using data from each game in relevant_nhl_games
        for game in range(len(relevant_nfl_games)):
            time.sleep(6)
            
            winning_team = relevant_nfl_games["winner/tie"].iloc[game]
            winning_abbr = nfl_name_conversions[nfl_name_conversions["Team Name"] == winning_team]["Team Abbreviation"].iloc[0]
            losing_team = relevant_nfl_games["loser/tie"].iloc[game]
            losing_abbr = nfl_name_conversions[nfl_name_conversions["Team Name"] == losing_team]["Team Abbreviation"].iloc[0]
            
            # Account for the strange labeling of urls when there is a neutral site game like the SB or international
            if relevant_nfl_games.iloc[game]["Unnamed: 5"] != "N":
                url_key = winning_abbr if relevant_nfl_games.iloc[game]["Unnamed: 5"] != "@" else losing_abbr
                url_insert = "".join(given_date.split("-")) + "0" + url_key.lower()
                game_url = "https://www.pro-football-reference.com/boxscores/" + url_insert + ".htm"
            
            else:
                webpage = requests.get(all_nfl_games_url)
                soup = BeautifulSoup(webpage.content, "html.parser")
                date_insert = "".join(given_date.split("-")) + "0"
                game_possibilities = soup.find_all("td", {"data-stat":"boxscore_word"})
                games_on_date = [x for x in game_possibilities if date_insert in str(x)]
                games_with_right_team = [x for x in games_on_date if winning_abbr.lower() in str(x) or losing_abbr.lower() in str(x)]
                addition = games_with_right_team[0].a["href"]
                game_url = "https://www.pro-football-reference.com" + addition
 
            
            # Using the built url, scrape Football Reference and grab the tables (we know the order of them) for offensive, defensive, special teams and snap counts
            game_tables = pd.read_html(game_url, header=1)
        
            
            offensive_game_stats = game_tables[2].rename(columns = {"Player":"player", "Tm":"team", "Cmp":"completions", "Att":"passing_attempts", "Yds":"passing_yards", "TD":"passing_touchdowns", 
                                 "Int":"interceptions", "Sk":"sacks_taken", "Yds.1":"sack_yards_taken", "Lng":"longest_pass", "Rate":"quaterback_rating", "Att.1":"rushing_attempts",
                                 "Yds.2":"rushing_yards", "TD.1":"rushing_touchdowns", "Lng.1":"longest_rush", "Tgt":"targets", "Rec":"receptions", "Yds.3":"receiving_yards",
                                 "TD.2":"receiving_touchdowns", "Lng.2":"longest_reception", "Fmb":"total_fumbles", "FL":"fumbles_lost"}).dropna(subset = ["player"])
            
            offensive_game_stats = offensive_game_stats[offensive_game_stats.player != "Player"]
            offensive_game_stats["passing_attempts"] = offensive_game_stats["passing_attempts"].apply(lambda x: int(x))
            offensive_game_stats["rushing_attempts"] = offensive_game_stats["rushing_attempts"].apply(lambda x: int(x))
            offensive_game_stats["receptions"] = offensive_game_stats["receptions"].apply(lambda x: int(x))
            
            point_tables = pd.read_html(game_url, header=0)
            point_table = point_tables[1].iloc[-1].iloc[-2:]
            offensive_game_stats["points_scored"] = offensive_game_stats.team.apply(lambda x: point_table.loc[x])
            offensive_game_stats["points_allowed"] = offensive_game_stats.team.apply(lambda x: point_table[point_table.index!=x][0])
            
            # Get and insert offensive and kicking IDs
            webpage = requests.get(game_url)
            soup = BeautifulSoup(webpage.content, "html.parser")
            
            offensive_ids = [str(x).split("data-append-csv=")[1].split(" ")[0].strip('"') for x in soup.find_all("tr") if "data-append-csv" in str(x)]
            offensive_names = [x.a.text.strip() for x in soup.find_all("tr") if "data-append-csv" in str(x)]
            offensive_id_dict = {offensive_names[n]:offensive_ids[n] for n in range(len(offensive_names))}
            offensive_game_stats["player_id"] = offensive_game_stats.player.apply(lambda z: offensive_id_dict[z])
            
            
            kicking_ids = [z.split('="')[1].split(" ")[0].strip('"') for z in [y for y in [x for x in soup.find_all(text=lambda text:isinstance(text, Comment)) if "kick" in str(x)] if "data-append-csv" in str(y)][1].split("data-append-csv")[1:]]
            kicking_names = [z.split("<")[0].strip() for z in [y for y in [x for x in soup.find_all(text=lambda text:isinstance(text, Comment)) if "kick" in str(x)] if "data-append-csv" in str(y)][1].split('.htm">')[1:]]
            kicking_id_dict = {kicking_names[n]:kicking_ids[n] for n in range(len(kicking_names))}
            
            defense_ids = [y.split('" ')[0].strip('"') for y in str([x for x in soup.find_all(text=lambda text:isinstance(text, Comment)) if "defense" in str(x)]).split("data-append-csv=")[1:]]
            defense_names = [y.split("<")[0].replace('\\', "").strip() for y in str([x for x in soup.find_all(text=lambda text:isinstance(text, Comment)) if "defense" in str(x)]).split('.htm">')[1:]]
            defense_id_dict = {defense_names[n]:defense_ids[n] for n in range(len(defense_names))}
            
            returns_ids = [y.split('" ')[0].strip('"') for y in str([x for x in soup.find_all(text=lambda text:isinstance(text, Comment)) if "returns" in str(x)]).split("data-append-csv=")[1:]]
            returns_names = [y.split("<")[0].replace('\\', "").strip() for y in str([x for x in soup.find_all(text=lambda text:isinstance(text, Comment)) if "returns" in str(x)]).split('.htm">')[1:]]
            returns_id_dict = {returns_names[n]:returns_ids[n] for n in range(len(returns_names))}
            
            
            # Make a table of all scored field goals and their distances from the pd.read_html. It doesn't really go with any category, but we'll need it for kicker scoring
            # Note: we need to do a new read_html because the header level is different than the original one
            scoring = pd.read_html(game_url, header=0)[1]
            field_goals = scoring[(scoring.Detail.str.contains("field goal") == True) & (scoring.Detail.str.contains("field goal return") == False)]
            field_goals["kicker"] = field_goals.Detail.apply(lambda x: " ".join(x.split("yard")[0].split(" ")[0:-2]))
            field_goals["distance"] = field_goals.Detail.apply(lambda x: (x.split("yard")[0].split(" ")[-2]))
            field_goals["date"] = str(month) + "/" + str(day) + "/" + str(year)
            final_field_goals = field_goals[["kicker", "distance", "date"]].copy()
            final_field_goal_df = final_field_goal_df.append(final_field_goals)
            
            # Make a column for if the team won
            offensive_game_stats["won"] = offensive_game_stats.team.apply(lambda x: 1 if x==winning_abbr else 0)

            # Make alterations to offensive stats - checking for division matchup, 
            teams = offensive_game_stats.team.unique()
            divisions = [nfl_name_conversions[nfl_name_conversions["Team Code"] == n]["League"].iloc[0] + nfl_name_conversions[nfl_name_conversions["Team Code"] == n]["Division"].iloc[0] for n in teams]
            offensive_game_stats["is_division_game"] = 1 if len(set(divisions)) == 1 else 0
            final_offensive_df = final_offensive_df.append(offensive_game_stats)
            
            
            # Other than the straightforward offensive stats, the other tables (defensive, kicking, etc.) are all hidden in comments. We now take a secondary route to find them
            scraped_html = requests.get(game_url)
            soup = BeautifulSoup(scraped_html.content)

            # Get all html comments, then filter out everything that isn't a table
            comments = soup.find_all(text=lambda text:isinstance(text, Comment))
            commented_out_tables = [BeautifulSoup(cmt).find_all('table') for cmt in comments]
            
            # Some of the entries in `commented_out_tables` are empty lists. Remove them.
            commented_out_tables = [tab[0] for tab in commented_out_tables if len(tab) == 1]

            # Go through the commented out tables pulling tables for kicking, defense, and snap counts, before altering titles and formatting to make them helpful
            for table in commented_out_tables:

                if table.get('id') == 'kicking':
                    kicking_table = pd.read_html(str(table), header=1)[0].dropna(subset=["Tm"]).rename(columns = {"Player":"player", "Tm":"team", "XPM":"extra_points_made", "XPA":"extra_points_attempted",
                                                                                                                  "FGM":"field_goals_made", "FGA":"field_goals_attempted", "Pnt":"punts",
                                                                                                                  "Yds":"punt_yards", "Y/P":"yards_per_punt", "Lng":"longest_punt"}).fillna(0)
                    kicking_table = kicking_table[kicking_table.team != "Tm"]
                    kicking_table["player_id"] = kicking_table.player.apply(lambda x: kicking_id_dict[x])
                    final_kicking_df = final_kicking_df.append(kicking_table)
                    
                if table.get("id") == "player_defense":
                    defense_table = pd.read_html(str(table), header=1)[0].dropna(subset=["Tm"]).rename(columns = {"Tm":"team", "Int":"interceptions", "Yds":"interception_return_yards",
                                                                                                      "TD":"interception_touchdowns", "Lng":"longest_interception_return",
                                                                                                      "Sk":"sacks", "Comb":"total_tackles", "TFL":"tackles_for_loss",
                                                                                                      "QBHits":"qb_hits", "FR":"fumble_recoveries", "Yds.1":"fumble_return_yards",
                                                                                                      "TD.1":"fumble_touchdowns"}).drop(columns = ["PD", "Solo", "Ast", "FF"])
                    defense_table = defense_table[defense_table.team != "Tm"]
                    defense_table["player_id"] = defense_table.Player.apply(lambda x: defense_id_dict[x])
                    final_defensive_df = final_defensive_df.append(defense_table)
                    
                    
                if table.get("id") == "returns":
                    returns_table = pd.read_html(str(table), header = 1)[0].dropna(subset=["Tm"]).rename(columns = {"Player":"player", "Tm":"team", "Yds":"kick_return_yards", "Yds.1":"punt_return_yards",
                                                                                                                    "TD":"kick_return_touchdowns", "TD.1":"punt_return_touchdowns"})
                    
                    returns_table = returns_table[returns_table.player != "Player"][["player", "team", "kick_return_yards", "kick_return_touchdowns", "punt_return_yards", "punt_return_touchdowns"]]
                    returns_table["player_id"] = returns_table.player.apply(lambda x: returns_id_dict[x])
                    final_returns_df = final_returns_df.append(returns_table)
      
                    
                if table.get("id") == "team_stats":
                    team_stats_table = pd.read_html(str(table), header = 0)[0].rename(columns = {"Unnamed: 0":"stat"})
                    penalty_yards = team_stats_table[team_stats_table.stat == "Penalties-Yards"].T.iloc[1:]
                    penalty_yards = penalty_yards[penalty_yards.columns[0]].apply(lambda x: float(x.split("-")[-1]))
                    final_penalty_df = pd.concat([final_penalty_df, penalty_yards])
                    
                if table.get("id") == "home_snap_counts" or table.get("id") == "vis_snap_counts":
                    location = table.get("id")

                    snaps = pd.read_html(str(table), header=1)[0].rename(columns = {"Player":"player", "Num":"offensive_snaps", "Pct":"offensive_percent", "Num.1":"defensive_snaps",
                                                                                        "Pct.1":"defensive_percent", "Num.2":"special_teams_snaps", "Pct.2":"special_teams_percent"})

                    snaps["total_offensive_snaps"] = snaps.apply(lambda x: x.offensive_snaps * (100/float(x.offensive_percent.split("%")[0])) if x.offensive_percent != "0%" else 0, axis=1)
                    snaps["total_offensive_snaps"] = round(snaps[snaps.offensive_snaps == max(snaps.offensive_snaps)].total_offensive_snaps.iloc[0])

                    snaps["total_defensive_snaps"] = snaps.apply(lambda x: x.defensive_snaps * (100/float(x.defensive_percent.split("%")[0])) if x.defensive_percent != "0%" else 0, axis=1)
                    snaps["total_defensive_snaps"] = round(snaps[snaps.defensive_snaps == max(snaps.defensive_snaps)].total_defensive_snaps.iloc[0])

                    snaps["total_special_teams_snaps"] = snaps.apply(lambda x: x.special_teams_snaps * (100/float(x.special_teams_percent.split("%")[0])) if x.special_teams_percent != "0%" else 0, axis=1)
                    snaps["total_special_teams_snaps"] = round(snaps[snaps.special_teams_snaps == max(snaps.special_teams_snaps)].total_special_teams_snaps.iloc[0])

                    snaps = snaps.drop(columns = ["Pos", "offensive_percent", "defensive_percent", "special_teams_percent"])
                    
                    final_snap_counts_df = final_snap_counts_df.append(snaps)

            final_field_goal_df["team"] = final_field_goal_df.kicker.apply(lambda x: final_kicking_df[final_kicking_df.player == x].team.iloc[0])
            
    

    return {"Offense":final_offensive_df.reset_index(drop=True), "Defense":final_defensive_df.reset_index(drop=True), "Kicking":final_kicking_df.reset_index(drop=True),
            "Snaps":final_snap_counts_df.reset_index(drop=True), "Field Goals":final_field_goal_df.reset_index(drop=True), "Returns":final_returns_df.reset_index(drop=True),
            "Penalties":final_penalty_df.rename(columns = {0:"penalty_yards"}).reset_index().rename(columns = {"index":"team"}), "date":given_date}

## MLB Stats Collection

In [16]:
def get_daily_mlb_stats(day, month, year):

    """A function that will return a cleaned pandas dataframe of daily MLB statistics for BOTH players and teams that played on the 
       given day.
       NOTE: All of day, month and year must be supplied as ints! 
       NOTE: This function can only be used for singular days, or the matching of location and opponent at the end of team data collection will return multiple/differing results, not singular locations and opponents!
       """ 
    def runs_allowed(row):
        return team_batting[(team_batting.game_id == row.game_id) & (team_batting["Team Name"] != row["Team Name"])].R.iloc[0]
    
    # if "{}/{}/{}".format(year, month, day) not in playing dates:
    #     return pd.DataFrame()
    
    given_date = str(year).zfill(2) + "-" + str(month).zfill(2) + "-" + str(day).zfill(2)
    
    dh_dict = {}
    
    # We will eventually return these dfs
    final_batting_df = pd.DataFrame()
    final_pitching_df = pd.DataFrame()
    final_team_df = pd.DataFrame()

    
    # Get all games played that day and their url inserts to be used in scraping baseball reference
    games = Boxscores(datetime(year, month, day)).games["{}-{}-{}".format(month, day, year)]
    
    for game in games:
        time.sleep(6)
        home_team = game["home_name"]
        
        if home_team not in dh_dict:
            dh_dict[home_team] = 1
        url_insert = mlb_name_conversions[mlb_name_conversions["Team Name"] == home_team]["Fangraphs Insert"].iloc[0]
        url = "https://www.fangraphs.com/boxscore.aspx?date={}&team={}&dh=0&season={}#home_standard".format(given_date, url_insert, year)
        
        try:
            tables = pd.read_html(url)
        except:
            try: 
                url = "https://www.fangraphs.com/boxscore.aspx?date={}&team={}&dh={}&season={}#home_standard".format(given_date, url_insert, dh_dict[home_team], year)
                tables = pd.read_html(url)
                dh_dict[home_team] += 1
            except:
                tables = pd.read_html(url)
        
        # Get and insert offensive and kicking IDs
        webpage = requests.get(url)
        soup = BeautifulSoup(webpage.content, "html.parser")
        
        all_ids = [y["href"].split("playerid=")[1].split("&")[0]for y in [item for sublist in [x.find_all("a") for x in soup.find_all("table", {"class":"rgMasterTable"})[0:4]] for item in sublist]]
        all_names = [y.text for y in [item for sublist in [x.find_all("a") for x in soup.find_all("table", {"class":"rgMasterTable"})[0:4]] for item in sublist]]
        
        id_dict = {all_names[n]:all_ids[n] for n in range(len(all_names))}
                
        
        # Get the basic info of the game like who played, how much they scored, and who won
        game_info = tables[14]
        home_team = game_info[0].iloc[2]
        home_runs = int(game_info[game_info.columns[-1]].iloc[2])
        away_team = game_info[0].iloc[1]
        away_runs = int(game_info[game_info.columns[-1]].iloc[1])
        winner = home_team if home_runs > away_runs else away_team
        innings = int(game_info.columns[0]) - 1
        
        game_id = home_team + str(home_runs) + away_team + str(away_runs) + given_date
        
        # Get the raw batting and pitching tables from the game
        home_batting = tables[27]
        home_pitching = tables[28]
        
        away_batting = tables[29]
        away_pitching = tables[30]
        
        
        # Combine the home and away stats for both batting and pitching
        home_batting["Team Name"] = home_team
        away_batting["Team Name"] = away_team
        
        home_pitching["Team Name"] = home_team
        away_pitching["Team Name"] = away_team
        
        batting = home_batting.append(away_batting)
        pitching = home_pitching.append(away_pitching)
        
        
        # Clean the batting tables before we append them to the final batting stats
        team_batting = batting[batting.Name == "Total"]
        player_batting = batting[batting.Name != "Total"]
        
        player_batting["total_bases"] = player_batting["1B"] + player_batting["2B"] * 2 + player_batting["3B"] * 3 + player_batting["HR"] * 4
        
        player_batting["Position"] = player_batting.Name.apply(lambda x: x.split(" - ")[-1])
        player_batting.Name = player_batting.Name.apply(lambda x: x.split(" - ")[0])

        player_batting["walks"] = player_batting.BB + player_batting.IBB + player_batting.HBP
        player_batting["sacrifices"] = player_batting.SF + player_batting.SH
        player_batting = player_batting.rename(columns = {"Name":"name", "PA":"plate_appearences", "H":"hits", "R":"runs", "1B":"singles", "2B":"doubles", "3B":"triples", "HR":"home_runs", "RBI":"runs_batted_in",
                                                          "SB":"stolen_bases", "SO":"strikeouts"})
        player_batting["game_id"] = game_id
        
       
        player_batting["player_id"] = player_batting.name.apply(lambda x: id_dict[x])
        
        
        # Clean the pitching tables before we append them to the final pitching stats
        pitching.Name = pitching.Name.apply(lambda x: x.split(" (")[0])
        
                # Right now we are calculating team pitching stats because it's easy, but I'm not including it in the output bc we don't have any stats with it. 
                # However, if we want to include them one day, here is where they are.
        team_pitching = pitching[pitching.Name == "Total"]
        player_pitching = pitching[pitching.Name != "Total"]
        
        player_pitching = player_pitching.rename(columns = {"Name":"name", "IP":"innings", "SV":"saves", "ER":"earned_runs", "SO":"Pitching Strikeouts"})
        player_pitching.innings = player_pitching.innings.apply(lambda x: float(str(x).split(".")[0] +"."+ str(int(str(x).split('.')[-1])/10*3.3).split(".")[-1]))
        player_pitching["game_id"] = game_id
        
        player_pitching["player_id"] = player_pitching.name.apply(lambda x: id_dict[x])
        
        # Next remove all the pitchers from the batting stats so NL Pitchers don't get batting stats
        pitchers_used = player_pitching.player_id
        player_batting = player_batting[player_batting.player_id.isin(pitchers_used) == False]
        
        
        # Clean the team tables before we append them to the final team stats
        team_batting["game_id"] = game_id
        team_batting["runs_allowed"] = team_batting.apply(lambda x: runs_allowed(x), axis = 1)
        team_batting["win_loss"] = team_batting.apply(lambda x: "W" if x.R > x.runs_allowed else "L", axis = 1)
            
        # Currently we assume all extra inning games are 10+ innings. If MLB keeps the double header rules then make sure to change to account for 8 inning extra inning games!!!
        team_batting["is_extra_innings"] = 1 if innings > 9 else 0
        team_batting = team_batting.rename(columns = {"Team Name":"team", "R":"runs_scored"})
        
        
        # Append the batting stats to the batting df and the picthing stats to the picthing df
        final_batting_df = final_batting_df.append(player_batting)
        final_pitching_df = final_pitching_df.append(player_pitching)
                # If we ever want to include team pitching stats, below is where we can include team_picthing
        final_team_df = final_team_df.append(team_batting)

        
        # Finally append the date to all dfs
        final_batting_df["Date"] = given_date
        final_pitching_df["Date"] = given_date
        final_team_df['Date'] = given_date
        
        final_team_df = final_team_df[['team'] + [col for col in final_team_df.columns if col not in ["team", "Name"]]]
        

        
    return {"batters":final_batting_df.reset_index(drop=True), "pitchers":final_pitching_df.reset_index(drop=True), "teams":final_team_df.reset_index(drop=True), "date":given_date}
    
    
    

# Transform Daily Stats

### Transform NBA Stats

In [17]:
def transform_nba_stats(nba):
    if len(nba) == 0:
        return pd.DataFrame()
    

    nba_stats = pd.DataFrame()
    team_stats = pd.DataFrame()
    
    nba_data = nba["players"]
    team_data = nba["teams"]
    date = nba["date"]
    
    # Build standard player stats
    nba_stats["Name"] = nba_data["Starters"]
    nba_stats["Team"] = nba_data["team"]
    nba_stats["Points"] = nba_data["PTS"].apply(lambda x: int(x))
    #nba_stats["field_goals_made"] = nba_data["made_field_goals"] 
    nba_stats["Three Pointers"] = nba_data["3P"].apply(lambda x: int(x))
    nba_stats["Rebounds"] = nba_data["TRB"].apply(lambda x: int(x))
    nba_stats["Minutes"] = nba_data["MP"].apply(lambda x: float(x))
    nba_stats["Assists"] = nba_data["AST"].apply(lambda x: int(x))
    nba_stats["Turnovers"] = nba_data["TOV"].apply(lambda x: int(x))
    nba_stats["Blocks"] = nba_data["BLK"].apply(lambda x: int(x))
    nba_stats['Steals'] = nba_data["STL"].apply(lambda x: int(x))
    nba_stats["Fouls"] = nba_data["PF"].apply(lambda x: int(x))
    nba_stats["Player ID"] = nba_data["id"]
    
    
    
    nba_stats["Free Throws Made"] = nba_data["FT"].apply(lambda x: int(x))
    nba_stats["Free Throw Percentage Denominator"] = nba_data["FTA"].apply(lambda x: int(x))
    
    
    
    nba_stats["teams_share_of_points_numerator"] = nba_stats["Points"].apply(lambda x: int(x))
    nba_stats["teams_share_of_points_denominator"] = nba_stats["Team"].apply(lambda x: int(team_data[team_data.team == x].PTS.iloc[0]))
    
    
    # Build standard team_stats
    team_stats["Team"] = team_data["team"]
    team_stats["Is Win"] = team_data["is_win"].apply(lambda x: True if x == 1 else False)
    team_stats["Points Scored"] = team_data["PTS"].apply(lambda x: int(x))
    team_stats["Points Allowed"] = team_data.team.apply(lambda x: int(team_data[team_data.opponent == x].PTS.iloc[0]))
    team_stats["Point Differential"] = team_stats["Points Scored"] - team_stats["Points Allowed"]
    team_stats = team_stats.drop(columns = ["Points Scored", "Points Allowed"])
    #team_stats["players_fouled_out"] = team_data["team"].apply(lambda x: len(nba_data[(nba_data.team == x) & (nba_data.personal_fouls >= 6)]))
    
    ### WILL CALCULATE POINTS/SCORE WHEN ASSIGNING TO OWNERSHIP FOR DATES. WILL PUT ALL LISTS OF 8 HIGHEST MINS INTO ONE LIST, THEN CALCULATE THE STDEV
    team_stats["Team Minutes Distribution"] = team_data["team"].apply(lambda x: [nba_stats[nba_stats.Team == x].sort_values(by = "Minutes").iloc[0:8]])
    
    
    # Attatch the date to the dfs for easier reference later on
    nba_stats["Date"] = date
    team_stats["date"] = date
    
    # Fillna
    nba_stats = nba_stats.fillna(0)
    team_stats = team_stats.fillna(0)
    
    # Final filter down to just 2023 specific scoring categories
    nba_stats = nba_stats[["Name", "Player ID", "Three Pointers", "Free Throws Made", "Rebounds", "Assists", "Steals", "Blocks", "Turnovers", "Points", "Date"]]
    
    return {"players":nba_stats, "teams":team_stats}

### Transform NHL Stats

In [18]:
def transform_nhl_stats(nhl_data):
    if len(nhl_data) == 0:
        return pd.DataFrame()
    
    goalie_data = nhl_data["Goalies"]
    skater_data = nhl_data["Skaters"][nhl_data["Skaters"].player.isin(goalie_data.player)==False]
    player_data = skater_data.append(goalie_data).reset_index(drop=True)
    team_data = nhl_data["Teams"]
    penalty_data = nhl_data["Penalties"]
    date = nhl_data["date"]
    
    player_stats = pd.DataFrame()
    team_stats = pd.DataFrame()
    
    # Build standard player stats
    player_stats["Name"] = player_data.player.reset_index(drop=True)
    player_stats["Team"] = player_data.team
    player_stats["Goals"] = player_data.goals
    player_stats["Assists"] = player_data.assists
    player_stats["Hockey Points"] = player_stats.Goals + player_stats.Assists
    player_stats["Penalty Minutes"] = player_data.penalty_minutes
    player_stats["Shots"] = player_data.shots
    player_stats["Hockey Hits"] = player_data.hits
    player_stats["Hockey Blocks"] = player_data.blocks
    player_stats["Plus Minus"] = player_data.plus_minus
    player_stats["Power Play Points"] = player_data.power_play_points
    player_stats["Bench Minutes"] = player_data.time_on_ice
    player_stats["Fights"] = player_data.player.apply(lambda x: len(penalty_data[(penalty_data.player == x) & (penalty_data.penalty == "Fighting")]))
    player_stats["Goals Against"] = player_data.goals_against.reset_index(drop = True) 
    player_stats["Shots Against"] = player_data.shots_against
    player_stats["Saves"] = player_data.shots_against - player_data.goals_against
    player_stats["Shutouts"] = player_data.shutouts
    player_stats["Player ID"] = player_data.player_id
    
    # Build the standard team stats
    team_stats["Team"] = team_data.team
    team_stats["Points"] = team_data.points
    team_stats["Goals Scored"] = team_stats.Team.apply(lambda x: player_data[player_data.team == x].team_goals_scored.iloc[0])
    team_stats["Goals Allowed"] = team_stats.Team.apply(lambda x: player_data[player_data.team == x].team_goals_allowed.iloc[0])
    team_stats["Is Win"] = team_stats["Goals Scored"] > team_stats["Goals Allowed"]
    team_stats = team_stats.drop(columns = ["Goals Scored", "Goals Allowed"])
    
    # Build the non-standard team stats
    team_stats["Special Teams Strength Numerator"] = team_data.power_play_goals + team_data.short_handed_goals
    team_stats["Special Teams Strength Denominator"] = team_data.goals
    
    
    # Attatch the date to the dfs for easier reference later on
    player_stats["Date"] = date
    team_stats["Date"] = date
    
    player_stats = player_stats[player_stats.Name != "Empty Net"]
    
    # Fillna
    player_stats = player_stats.fillna(0)
    team_stats = team_stats.fillna(0)
    
    # Final filter down to just 2023 specific scoring categories
    player_stats = player_stats[["Name", "Player ID", "Hockey Points", "Hockey Hits", "Hockey Blocks", "Penalty Minutes", "Shots", "Saves", "Goals Against", "Date"]]
    

    
    return {"players":player_stats, "teams":team_stats}

### Transform NFL Stats

In [80]:
def transform_nfl_stats(nfl_data):
    if len(nfl_data) == 0:
        return pd.DataFrame()
    
    # Functions that will be necessary in transforming nfl stats
    # def return_stats(player, stat):
    #         try:
    #             ans = return_data[return_data.Player == player][stat].iloc[0]
    #             return float(ans)
    #         except:
    #             return 0
            
        
    # Pull in the raw data from the get_daily_xxx_stats function and clean as necessary
    offensive_data = nfl_data["Offense"]
    return_data = nfl_data["Returns"]
    offensive_data = offensive_data.join(return_data.set_index("player"), on = "player",how = "outer", rsuffix = "off")
    return_teams = offensive_data.teamoff
    offensive_data.team.update(return_teams)
    offensive_data = offensive_data.drop(columns = ["teamoff"])
    
    # Keep pulling in the raw data from the get_daily_xxx_stats function
    defensive_data = nfl_data["Defense"]
    kicking_data = nfl_data["Kicking"]
    snaps_data = nfl_data["Snaps"]
    field_goal_data = nfl_data["Field Goals"]
    penalty_data = nfl_data["Penalties"]
    date = nfl_data["date"]
    
    # Create dfs for data to end up in
    offensive_stats = pd.DataFrame()
    defensive_stats = pd.DataFrame()
    
    
    # Convert numbers that need to be calculated on into ints from the strings they are now
    for stat in ["rushing_yards", "rushing_touchdowns", "receiving_yards", "receiving_touchdowns", "total_fumbles", "fumbles_lost", "sacks", "passing_yards", "passing_touchdowns",
                 "interceptions", "fumble_recoveries", "interception_return_yards", "fumble_return_yards"] + ["kick_return_yards", "kick_return_touchdowns", "punt_return_yards", "punt_return_touchdowns"]:
        try:
            offensive_data[stat] = offensive_data[stat].apply(lambda x: float(x))
            defensive_data[stat] = defensive_data[stat].apply(lambda x: float(x))
        except:
            try:
                offensive_data[stat] = offensive_data[stat].apply(lambda x: float(x))
            except:
                defensive_data[stat] = defensive_data[stat].apply(lambda x: float(x))
                      
    # Build standard offensive stats
    offensive_stats["Name"] = offensive_data.player
    offensive_stats["Player ID"] = offensive_data.player_id
    offensive_stats["Team"] = offensive_data.team
    offensive_stats["All Purpose Yards"] = offensive_data.rushing_yards * 1.05 + offensive_data.passing_yards * 0.27 + offensive_data.receiving_yards
    offensive_stats["All Purpose TDs"] = offensive_data.rushing_touchdowns * .95 + offensive_data.receiving_touchdowns + offensive_data.passing_touchdowns * 0.32
    offensive_stats["Attempts"] = offensive_data.passing_attempts * .18 + offensive_data.rushing_attempts * .42 + offensive_data.receptions

    
    # Build standard return stats
    for return_stat in ["kick_return_yards", "kick_return_touchdowns", "punt_return_yards", "punt_return_touchdowns"]:
        offensive_stats[return_stat] = offensive_data[return_stat]
        
        
    offensive_stats["Return Yards"] = offensive_stats.kick_return_yards + offensive_stats.punt_return_yards * 2.49
    offensive_stats = offensive_stats.drop(columns = ["kick_return_yards", "kick_return_touchdowns", "punt_return_yards", "punt_return_touchdowns"])
    
    # Build standard defensive stats
    defensive_stats["player"] = defensive_data.team
    defensive_stats["Player ID"] = defensive_data.team
    defensive_stats["Sacks/Turnovers"] = defensive_data.sacks * 0.51 + defensive_data.interceptions + defensive_data.fumble_recoveries
    defensive_stats = defensive_stats.groupby(by = "Player ID").sum()
    
    # Build standard kicking stats
    kicking_data.punt_yards = kicking_data.punt_yards.apply(lambda x: float(x))
    field_goal_data.distance = field_goal_data.distance.apply(lambda x: float(x))
    punting_stats = pd.DataFrame()
    placekicking_stats = pd.DataFrame()
    
    punting_stats["Name"] = kicking_data[kicking_data.punt_yards >0].player.unique()
    punting_stats["Player ID"] = punting_stats.Name.apply(lambda x: kicking_data[kicking_data.player == x].player_id.iloc[0])
    punting_stats["Team"] = punting_stats.Name.apply(lambda x: kicking_data[kicking_data.player == x].team.iloc[0])
    placekicking_stats["Name"] = field_goal_data.kicker.unique()
    placekicking_stats["Player ID"] = placekicking_stats.Name.apply(lambda x: kicking_data[kicking_data.player == x].player_id.iloc[0])
    placekicking_stats["Team"] = placekicking_stats.Name.apply(lambda x: kicking_data[kicking_data.player == x].team.iloc[0])
    
    punting_stats["Punt Yards"] = punting_stats.Name.apply(lambda x: kicking_data[kicking_data.player == x].punt_yards.sum())
    placekicking_stats["Field Goal Yards"] = placekicking_stats.Name.apply(lambda x: field_goal_data[field_goal_data.kicker == x].distance.sum())
    total_kicking_stats = pd.concat([punting_stats, placekicking_stats])
    total_kicking_stats["Punt Yards"] = total_kicking_stats["Punt Yards"].apply(lambda x: 0 if pd.isna(x) == True else x)
    total_kicking_stats["Field Goal Yards"] = total_kicking_stats["Field Goal Yards"].apply(lambda x: 0 if pd.isna(x) == True else x)
    total_kicking_stats["Kicking Yards"] = total_kicking_stats["Punt Yards"] * .4 + total_kicking_stats["Field Goal Yards"]
    total_kicking_stats = total_kicking_stats.drop(columns = ["Punt Yards", "Field Goal Yards"])  
    
    individual_stats = offensive_stats.append(total_kicking_stats)
    
    # Attatch the date to each df for easier reference later on
    individual_stats["Date"] = date
    defensive_stats["Date"] = date
    
    # Fillna
    individual_stats = individual_stats.fillna(0)
    defensive_stats = defensive_stats.fillna(0)
    
    # We are currently unable to assign kickers to a specific team
    individual_stats.team = individual_stats.Team.apply(lambda x: "Unknown" if x == 0 else x)
    
    individual_stats = individual_stats.drop(columns = ["Team"])

    
    return {"players":individual_stats.reset_index(drop = True), "teams":defensive_stats.reset_index(drop = False)}
    

### Transform MLB Stats

In [60]:
def transform_mlb_stats(mlb_data):
    batting_data = mlb_data["batters"]
    pitching_data = mlb_data["pitchers"]
    team_data= mlb_data["teams"]
    date = mlb_data["date"]
    
    batting_stats = pd.DataFrame()
    pitching_stats = pd.DataFrame()
    team_stats = pd.DataFrame()
    
    # Build standard batting_stats
    batting_stats["Name"] = batting_data.name
    batting_stats["Plate Appearences"] = batting_data.plate_appearences
    batting_stats["Hits"] = batting_data.hits
    batting_stats["Walks"] = batting_data.walks
    batting_stats["Runs"] = batting_data.runs
    batting_stats["RBIs"] = batting_data.runs_batted_in
    batting_stats["XBH"] = batting_data.doubles + batting_data.triples + batting_data.home_runs
    batting_stats["Home Runs"] = batting_data.home_runs
    batting_stats["Stolen Bases"] = batting_data.stolen_bases
    batting_stats["Sacrifices"] = batting_data.sacrifices
    batting_stats["Strikeouts"] = batting_data.strikeouts
    batting_stats["Player ID"] = batting_data.player_id.apply(lambda x: int(x))
    batting_stats["Total Bases"] = batting_data.total_bases
    batting_stats["Date"] = batting_data.Date
   
    
    # Build the standard pitching stats
    pitching_stats["Name"] = pitching_data.name
    pitching_stats["Innings"] = pitching_data.innings.apply(lambda x: float(x))
    pitching_stats["Saves"] = pitching_data.saves
    pitching_stats["Player ID"] = pitching_data.player_id.apply(lambda x: int(x))
    pitching_stats["Pitching Strikeouts"] = pitching_data["Pitching Strikeouts"].apply(lambda x: int(x))
    pitching_stats["Date"] = pitching_data.Date
    
    # Build the non-standard pitching stats
    pitching_stats["Earned Runs"] = pitching_data.earned_runs.apply(lambda x: int(x))
    pitching_stats["ERA Denominator"] = pitching_data.innings
    

    
    # Build standard team stats
    team_stats["Team"] = team_data.team
    team_stats["Is Win"] = team_data.win_loss.apply(lambda x: True if x == "W" else False)
    team_stats["Run Differential"] = team_data.runs_scored - team_data.runs_allowed
    team_stats["Is Extra Inning Game"] = team_data.is_extra_innings.apply(lambda x: True if x == 1 else False)
    
    
    # Combine the batting and pitching stats into one df for all players
    player_stats = pd.concat([batting_stats, pitching_stats])
    player_stats = player_stats.fillna(0)
  
    # Final Filer down to only 2023 specific scoring stats
    player_stats = player_stats[["Name", "Player ID", "Runs", "Home Runs", "RBIs", "Stolen Bases", "Walks", "Strikeouts", "Total Bases", "Innings", "Earned Runs", "Pitching Strikeouts", "Saves", "Date"]]
    
    
    return {"players":player_stats, "teams":team_stats}
    

# Build the Waiver Wire

##### Helper Functions

In [61]:
def check_valid_team(roster):
    """ Given a roster and its players, checks that the team meets all requirements. Can be used in testing for completing a 
        waiver claim"""
    
    valid = True
    
    total_players = roster[roster.Position != "Injured Reserve"]
    valid = False if len(total_players) != 29 else valid
    
    
    # Check that each league has the correct number of players
    mlb_players = roster[roster.League == "MLB"]
    valid = False if len(mlb_players) != 10 else valid

    
    nfl_players = roster[roster.League == "NFL"]
    valid = False if len(nfl_players) != 8 else valid
 
    nhl_players = roster[roster.League == "NHL"]
    valid = False if len(nhl_players) != 6 else valid
    
    nba_players = roster[roster.League == "NBA"]
    valid = False if len(nba_players) != 5 else valid


    # Check that each position within each league has the correct number of players
    mlb_roster = roster[roster.League == "MLB"]
    valid = False if len(mlb_roster[mlb_roster.Position == "Pitcher"]) != 2 else valid
    valid = False if max([len(mlb_roster[mlb_roster.Position == x]) for x in ["Catcher", "First Base", "Second Base", "Short Stop", "Third Base", "Short Stop", "Left Field", "Center Field", "Right Field"]]) > 1 else valid
    
    nfl_roster = roster[roster.League == "NFL"]
    valid = False if max([len(nfl_roster[nfl_roster.Position == x]) for x in ["Quarterback", "Running Back", "Wide Receiver", "Tight End", "Kicker", "Punter", "Returner", "Defense"]]) > 1 else valid
    
    nhl_roster = roster[roster.League == "NHL"]
    valid = False if len(nhl_roster[nhl_roster.Position == "Defenseman"]) != 2 else valid
    valid = False if max([len(nhl_roster[nhl_roster.Position == x]) for x in ["Goalie", "Center", "Right Wing", "Left Wing"]]) > 1 else valid
    
    nba_roster = roster[roster.League == "NBA"]
    valid = False if max([len(nba_roster[nba_roster.Position == x]) for x in ["Point Guard", "Shooting Guard", "Small Forward", "Power Forward", "Center"]]) > 1 else valid
    
    valid = False if max([len(roster[roster.Position == x]) for x in ["Injured Reserve"]]) > 1 else valid
    return valid

In [146]:
def claim_id_checker(league, id_value, player_name):
    league = "mlb" if league == "MLB" else "nfl" if league == "NFL" else "nhl" if league == "NHL" else "NBA" if league == "NBA" else "Failure"
    if league == "Failure":
        failure = "Invalid League for Claim"
        return failure
        
    elif league == "mlb":
        url = "https://www.baseball-reference.com/players/{}/{}.shtml".format(id_value[0], id_value)
        
    elif league == "nfl":
        url = "https://www.pro-football-reference.com/players/{}/{}.htm".format(id_value[0], id_value)
        
    elif league == "nhl":
        url = "https://www.hockey-reference.com/players/{}/{}.html".format(id_value[0], id_value)
        
    elif league == "nba":
        url = "https://www.basketball-reference.com/players/{}/{}.html".format(id_value[0], id_value)
  
    webpage = requests.get(url)
    soup = BeautifulSoup(webpage.content, "html.parser")

    valid_pickup_id_check = False if player_name not in str(soup.find("title")) else True
    claim_success = False if valid_pickup_id_check == False else True
    failure = "Invalid ID for Pickup Player" if valid_pickup_id_check == False else "Successful"
    
    return failure
    

### Main Functions

In [64]:
def complete_waivers(waiver_sheet_file_path):

    all_current_rostered_players = pd.DataFrame()
    current_rosters = {}
    all_requested_waiver_claims = {}
    injured_reserve_moves = {}
    waiver_history = pd.DataFrame(columns = ["Participant", "Action"])
    ir_waiver_history = pd.DataFrame(columns = ["Participant", "Action"])

    for participant in waiver_participants:
        waiver_claims = pd.read_excel(waiver_sheet_file_path, sheet_name = participant, header = 4)
        current_roster = waiver_claims.iloc[:, 1:5]
        all_current_rostered_players = all_current_rostered_players.append(current_roster)
        current_rosters[participant] = current_roster

        requested_waiver_claims = waiver_claims.iloc[0:15, 7:].dropna(how = "all")
        all_requested_waiver_claims[participant] = requested_waiver_claims

        injured_reserve_info = waiver_claims.iloc[17:, 6:]
        injured_reserve_moves[participant] = injured_reserve_info.reset_index(drop = True)

    # Determine the waiver priority, and cross reference with only the participants who made claims this cycle
    participants_with_claims = [x for x in all_requested_waiver_claims if len(all_requested_waiver_claims[x]) > 0]
    total_waiver_priority = ["Arron"] ## Actually Determine Waiver Priority Based on Last Weeks Standings
    waiver_priority = [x for x in total_waiver_priority if x in participants_with_claims]

    while max([len(all_requested_waiver_claims[x]) for x in waiver_priority], default = 0) > 0:
        top_priority = waiver_priority[0]
        claim = all_requested_waiver_claims[top_priority].iloc[0]
        claim_success = True

        # Check that there is an ID for both a pickup and a dropping player
        ids = pd.Series([claim["ID.1"], claim["ID.2"]])
        ids_check = True if len(ids[ids.isna() == False]) == 2 else False
        claim_success = False if ids_check == False else claim_success
        failure = "Fewer than 2 IDs inputted in the transaction" if ids_check == False else ""

        # Check if the ID of the player requested to pickup is valid
        id_check = claim_id_checker(claim["League.1"], claim["ID.1"], claim["Player.1"])
        claim_success = False if id_check != "Successful" else True
        failure = "Player Claim has an Invalid ID" if id_check == False else failure

        # Next, check that the ID of the player requested to drop is on the active roster
        valid_dropping_id_check = False if claim["ID.2"] not in current_rosters[top_priority].ID.values else True
        claim_success = False if valid_dropping_id_check == False else claim_success
        failure = "Invalid ID for Dropping Player" if valid_dropping_id_check == False else failure


        # Maybe later include a check if the position of the pick up was valid

        # Next, check that the pickup player is not on an active roster
        player_availability_check = True if claim["ID.1"] not in all_current_rostered_players.ID.values else False
        claim_success = False if player_availability_check == False else claim_success
        failure = "Player Is Already Rostered On Another Team" if player_availability_check == False else failure


        # Finally, check that completing the transaction would result in a valid team
        team_copy = current_rosters[top_priority].copy()
        theoretical_drop = team_copy[team_copy.ID != claim["ID.2"]]
        theoretical_add = theoretical_drop.append(pd.DataFrame(claim.iloc[0:4].rename({x:x.split(".")[0] for x in claim.iloc[0:4].index})).T).reset_index(drop = True)
        legal_team_check = check_valid_team(theoretical_add)
        claim_success = False if legal_team_check == False else claim_success
        failure = "The Roster Would Not Be Legal If the Transaction Went Through" if legal_team_check == False and valid_dropping_id_check != False else failure

        # If the claim is still successfull, complete it, add the added player to the current roster list (so someone can't grab them this week too), update top priority, and move on
        if claim_success == True:
            # add the claim, and drop the player
            current_rosters[top_priority] = current_rosters[top_priority].append(pd.DataFrame(claim.iloc[0:4].rename({x:x.split(".")[0] for x in claim.iloc[0:4].index})).T).reset_index(drop = True)
            current_rosters[top_priority] = current_rosters[top_priority][current_rosters[top_priority].ID != claim["ID.2"]].reset_index(drop = True)

            # add the player to all currently rostered players
            all_current_rostered_players = all_current_rostered_players.append(pd.DataFrame(claim.iloc[0:4].rename({x:x.split(".")[0] for x in claim.iloc[0:4].index})).T).reset_index(drop = True)

            # remove the completed claim from top priorities claims list
            all_requested_waiver_claims[top_priority] = all_requested_waiver_claims[top_priority].iloc[1:] if len(all_requested_waiver_claims[top_priority]) > 1 else pd.DataFrame()

        else:
            # remove the completed claim from top priorities claims list
            all_requested_waiver_claims[top_priority] = all_requested_waiver_claims[top_priority].iloc[1:] if len(all_requested_waiver_claims[top_priority]) > 1 else pd.DataFrame()

        # update top priority and remove a participant if they no longer have waiver claims
        waiver_priority.append(waiver_priority.pop(waiver_priority.index(top_priority)))
        num_claims = len(all_requested_waiver_claims[top_priority])
        if num_claims == 0:
            waiver_priority.remove(top_priority)
            del all_requested_waiver_claims[top_priority]

        # Fill in the waiver history sheet to be distributed
        if claim_success == True:
            waiver_history = waiver_history.append(pd.Series({"Participant":top_priority, "Action":"Succesfully claimed {} and dropped {}".format(claim["Player.1"], claim["Player.2"])}), ignore_index = True)
        else:
            waiver_history = waiver_history.append(pd.Series({"Participant":top_priority, "Action":"Failed to claim {}. Reason: {}".format(claim["Player.1"], failure)}), ignore_index = True)


        # Now Tackle IR Claims (Note: regular claims will take precedent over IR claims - thus the 2nd choice on IR claims.
        participants_with_ir_claims = [x for x in injured_reserve_moves if injured_reserve_moves[x]["Claim No."].iloc[2] == "Yes" or injured_reserve_moves[x]["League.2"].iloc[2]]
        ir_priority = [x for x in total_waiver_priority if x in participants_with_ir_claims]

        # Simply loop through the IR waiver wire, as participants can only make 1 claim at a time
        for participant in ir_priority:
            adding_a_player = "Yes" if injured_reserve_moves[participant]["Claim No."].iloc[2] == "Yes" else "No"
            if adding_a_player == "Yes":
                claim = injured_reserve_moves[participant].copy()

                # Get the info for both the player moving to IR and the player the participant is looking to claim
                player_to_add = claim["Player.1"].iloc[5]
                id_to_add = claim["ID.1"].iloc[5]

                player_to_claim = claim["Player.1"].iloc[8]
                id_to_claim = claim["ID.1"].iloc[8]

                # Check that the player to add to IR is on the current roster
                current_roster_check = False if claim["ID.1"].iloc[5] not in current_rosters[participant].ID.values else True
                ir_success = True if current_roster_check == True else False
                ir_failure = "The player to add to IR is not currently on the active roster" if current_roster_check == False else "The Injured Reserve Move was Succesfull"

                # Check that the player to add to the team has a valid id, and if not, move to the second choice
                ir_id_check = claim_id_checker(claim["League.1"].iloc[8], claim["ID.1"].iloc[8], claim["Player.1"].iloc[8])
                ir_success = False if ir_id_check != "Successful" else True
                ir_failure = "Injured Reserve Player Claim has an Invalid ID" if ir_id_check != "Success" else failure

                secondary_selection = False
                if ir_id_check != "Successful":
                    secondary_selection = True
                    ir_id_check = claim_id_checker(claim["League.1"].iloc[9], claim["ID.1"].iloc[9], claim["Player.1"].iloc[9])
                    ir_success = False if ir_id_check != "Successful" else True
                    ir_failure = "Both Injured Reserve Players Claims have Invalid IDs" if ir_id_check != "Success" else failure

                row = 9 if secondary_selection == True else 8


                # Next, check that the pickup player is not on an active roster
                ir_player_availability_check = True if claim["ID.1"].iloc[row] not in all_current_rostered_players.ID.values else False
                ir_success = False if ir_player_availability_check == False else ir_success
                ir_failure = "Injured Reserve Claim Player Is Already Rostered On Another Team" if ir_player_availability_check == False else ir_failure

                if ir_player_availability_check == False:
                    secondary_selection = True
                    ir_player_availabilit_check = True if claim["ID.1"].iloc[9] not in all_current_rostered_players.ID.values else False
                    ir_success = False if ir_player_availability_check == False else True
                    ir_failure = "The Injured Reserve Claim Player is Already Rostered On Another Team" if ir_player_availability_check == False else failure

                row = 9 if secondary_selection == True else 8


                #BEFORE THE NEXT PART, WE SHOULD CHECK IF THE ADDING A NEW PLAYER IS LEFT BLANK, AND IF IT IT, IF THE RETURNING IR
                # TO THE TEAM IS A YES. THEN IF SO, ADD THE OLD IR PERSON BACK ONTO THE TEAM
                current_ir_return = "No"
                if pd.isna(claim["ID.1"].iloc[8]) == True and pd.isna(claim["ID.1"].iloc[9]) == True:
                    ir_failure = "The Transaction Was Successfull"
                    ir_success = True

                    current_ir_return = claim["League.2"].iloc[4]
                    if current_ir_return == "Yes":
                        #check that completing the transaction would result in a valid team
                        ir_team_copy = current_rosters[participant].copy()
                        ir_theoretical_move = ir_team_copy.copy()

                        old_ir_spot = ir_theoretical_move[ir_theoretical_move.Position == "Injured Reserve"]

                        # Slide the old IR player back onto the roster
                        ir_theoretical_move = ir_theoretical_move[ir_theoretical_move.Position != "Injured Reserve"]
                        ir_theoretical_move = ir_theoretical_move.append(pd.DataFrame.from_dict({"League":[claim["Position.2"].iloc[7]], "Position":[claim["Player.2"].iloc[7]], "Player":[claim["ID.2"].iloc[7]], "ID":[claim["Unnamed: 15"].iloc[7]]})).reset_index(drop = True)

                        # Slide the old healthy player into the IR
                        ir_theoretical_move = ir_theoretical_move[ir_theoretical_move.ID != claim["ID.1"].iloc[5]]
                        ir_theoretical_move = ir_theoretical_move.append(pd.DataFrame.from_dict({"League":[np.NaN], "Position":["Injured Reserve"], "Player":[claim["Player.1"].iloc[5]], "ID":[claim["ID.1"].iloc[5]]})).reset_index(drop = True)
                        z = ir_theoretical_move
                        ir_legal_team_check = check_valid_team(ir_theoretical_move)
                        ir_success = False if ir_legal_team_check == False else ir_success
                        ir_failure = "The Roster Would Not Be Legal If the Transaction Went Through" if ir_legal_team_check == False else failure

                        # If the IR claim is still successfull, complete it, add the added player to the current roster list (so someone can't grab them this week too)
                        if ir_success == True:
                            # add the claim, and move the player
                            current_rosters[participant] = ir_theoretical_move.copy()

                    else:
                        ir_failure = "The Roster Would Not Be Legal If the Transaction Went Through"

                else:
                    # Finally, check that completing the transaction would result in a valid team
                    ir_team_copy = current_rosters[participant].copy()
                    ir_theoretical_move = ir_team_copy.copy()

                    old_ir_spot = ir_theoretical_move[ir_theoretical_move.Position == "Injured Reserve"]
                    ir_theoretical_move = ir_theoretical_move[ir_theoretical_move.Position != "Injured Reserve"]

                    ir_theoretical_move = ir_theoretical_move[ir_theoretical_move.ID != claim["ID.1"].iloc[5]]
                    ir_theoretical_move = ir_theoretical_move.append(pd.DataFrame.from_dict({"League":[np.NaN], "Position":["Injured Reserve"], "Player":[claim["Player.1"].iloc[5]], "ID":[claim["ID.1"].iloc[5]]})).reset_index(drop = True)

                    ir_theoretical_add = ir_theoretical_move.append(pd.DataFrame.from_dict({"League":[claim["League.1"].iloc[row]], "Position":[claim["Position.1"].iloc[row]], "Player":[claim["Player.1"].iloc[row]], "ID":[claim["ID.1"].iloc[row]]})).reset_index(drop = True)
                    ir_legal_team_check = check_valid_team(ir_theoretical_add)
                    ir_success = False if ir_legal_team_check == False else ir_success
                    ir_failure = "The Roster Would Not Be Legal If the Transaction Went Through" if ir_legal_team_check == False else failure


                    # If the IR claim is still successfull, complete it, add the added player to the current roster list (so someone can't grab them this week too)
                    if ir_success == True:
                        # add the claim, and drop the player
                        current_rosters[participant] = current_rosters[participant][current_rosters[participant].Position != "Injured Reserve"].reset_index(drop = True)
                        current_rosters[participant] = current_rosters[participant][current_rosters[participant].ID != claim["ID.1"].iloc[5]].reset_index(drop = True)

                        current_rosters[participant] = current_rosters[participant].append(pd.DataFrame.from_dict({"League":[np.NaN], "Position":["Injured Reserve"], "Player":[claim["Player.1"].iloc[5]], "ID":[claim["ID.1"].iloc[5]]})).reset_index(drop = True)
                        current_rosters[participant] = current_rosters[participant].append(pd.DataFrame.from_dict({"League":[claim["League.1"].iloc[row]], "Position":[claim["Position.1"].iloc[row]], "Player":[claim["Player.1"].iloc[row]], "ID":[claim["ID.1"].iloc[row]]})).reset_index(drop = True)


                        # add the player to all currently rostered players
                        all_current_rostered_players = all_current_rostered_players.append(pd.DataFrame.from_dict({"League":[claim["League.1"].iloc[row]], "Position":[claim["Position.1"].iloc[row]], "Player":[claim["Player.1"].iloc[row]], "ID":[claim["ID.1"].iloc[row]]})).reset_index(drop = True)


                # Fill in the waiver history sheet to be distributed
                if ir_success == True:
                    ir_waiver_history = ir_waiver_history.append(pd.Series({"Participant":participant, "Action":"Succesfully moved {} to Injured Reserve and added {} back to team".format(claim["Player.1"].iloc[5], claim["Player.1"].iloc[row] if current_ir_return != "Yes" else claim["ID.2"].iloc[7])}), ignore_index = True)
                else:
                    ir_waiver_history = ir_waiver_history.append(pd.Series({"Participant":participant, "Action":"Failed to move {} to Injured Reserve. Reason: {}".format(claim["Player.1"].iloc[5], ir_failure)}), ignore_index = True)



            if injured_reserve_moves[participant]["Claim No."].iloc[2] == "No" and injured_reserve_moves[participant]["League.2"].iloc[2] == "Yes":
                old_ir = current_rosters[participant][current_rosters[participant].Position == "Injured Reserve"].copy()
                claim = injured_reserve_moves[participant].copy()
                simple_drop = True if claim["League.2"].iloc[4] == "No" else "False"
                if simple_drop == True:
                    ir_success = True
                    current_rosters[participant] = current_rosters[participant][current_rosters[participant].Position != "Injured Reserve"].reset_index(drop = True)
                    current_rosters[participant] = current_rosters[participant].append(pd.DataFrame.from_dict({"League":[np.NaN], "Position":["Injured Reserve"], "Player":["Empty"], "ID":[np.NaN]})).reset_index(drop = True)    

                else:
                    ir_theoretical_team = current_rosters[participant].copy()

                    # Drop the player listed to drop
                    ir_theoretical_team = ir_theoretical_team[ir_theoretical_team.ID != claim["Unnamed: 15"].iloc[10]]

                    # Add the player on IR back to the team
                    ir_theoretical_team = ir_theoretical_team.append(pd.DataFrame.from_dict({"League":[claim["Position.2"].iloc[7]], "Position":[claim["Player.2"].iloc[7]], "Player":[claim["ID.2"].iloc[7]], "ID":[claim["Unnamed: 15"].iloc[7]]})).reset_index(drop = True)

                    ir_legal_team_check_ii = check_valid_team(ir_theoretical_team)
                    ir_success_ii = False if ir_legal_team_check_ii == False else True
                    ir_failure_ii = "The Roster Would Not Be Legal If the Transaction Went Through" if ir_legal_team_check_ii == False else "Success"

                    if ir_success_ii == True:
                        # Remove the old IR player, and add him back in his actual position
                        current_rosters[participant] = current_rosters[participant][current_rosters[participant].Position != "Injured Reserve"].reset_index(drop = True)
                        current_rosters[participant] = current_rosters[participant].append(pd.DataFrame.from_dict({"League":[claim["Position.2"].iloc[7]], "Position":[claim["Player.2"].iloc[7]], "Player":[claim["ID.2"].iloc[7]], "ID":[claim["Unnamed: 15"].iloc[7]]})).reset_index(drop = True)

                        # Remove the player manually listed for a drop given the addition of the IR Player
                        current_rosters[participant] = current_rosters[participant][current_rosters[participant].ID != claim["Unnamed: 15"].iloc[10]]

                        # Add back an empty IR spot
                        current_rosters[participant] = current_rosters[participant].append(pd.DataFrame.from_dict({"League":[np.NaN], "Position":["Injured Reserve"], "Player":["Empty"], "ID":[np.NaN]})).reset_index(drop = True)

                # Fill in the waiver history sheet to be distributed
                if ir_success_ii == True:
                    if simple_drop == True:
                        ir_waiver_history = ir_waiver_history.append(pd.Series({"Participant":participant, "Action":"Succesfully Removed {} From The Injured Reserve".format(old_ir.Player.iloc[0])}), ignore_index = True)
                    else:
                        ir_waiver_history = ir_waiver_history.append(pd.Series({"Participant":participant, "Action":"Succesfully Added {} Back From The Injured Reserve and Dropped {}".format(old_ir.Player.iloc[0], claim["ID.2"].iloc[10])}), ignore_index = True)

                else:
                    ir_waiver_history = ir_waiver_history.append(pd.Series({"Participant":participant, "Action":"Failed to remove {} to Injured Reserve. Reason: {}".format(claim["ID.2"].iloc[6], ir_failure_ii)}), ignore_index = True)

    waiver_history = waiver_history.append(ir_waiver_history)
    
    return current_rosters, waiver_history

# Functions to run the League

##### Helper Functions

In [65]:
def assign_fantasy_points(raw_stats_line, league, points_dict, postitional_dict):
    raw_stats_line = raw_stats_line.fillna(0)
    position = raw_stats_line.Position
    positional_scoring_categories = positional_scoring_dict[league][position]
    total_points = 0
    for col in raw_stats_line.index:
        if col in positional_scoring_categories:
            total_points += raw_stats_line[col] * points_dict[league][col]
    return round(total_points, 2)

In [66]:
# Get current full rosters and starting 9s from the Current Rosters Excel
def get_current_rosters(rosters_file_path):
    rosters = {}
    for participant in waiver_participants:
        rosters[participant] = {}

        data = pd.read_excel(rosters_file_path, sheet_name = participant)

        # Grab the full current roster and store it in the rosters dictionary
        full_roster = data.iloc[4:, 1:5]
        full_roster.columns = ["League", "Position", "Player", "ID"]
        rosters[participant]["Full Roster"] = full_roster.dropna(how = "all").reset_index(drop = True)

        # Grab the starters and store it in the rosters dictionary
        starters = data.iloc[5:, 7:]
        starters.columns = ["League", "Position", "Player", "ID"]
        rosters[participant]["Starters"] = starters.dropna(how = "all").reset_index(drop = True)

    return rosters

### Main Functions

In [67]:
def get_daily_league_stats(day, month, year):
    raw_nfl_stats = get_daily_nfl_stats(day, month, year)
    nfl_stats = transform_nfl_stats(raw_nfl_stats)
    time.sleep(20)
    
    raw_nhl_stats = get_daily_nhl_stats(day, month, year)
    nhl_stats = transform_nhl_stats(raw_nhl_stats)
    time.sleep(20)
    
    raw_nba_stats = get_daily_nba_stats(day, month, year)
    nba_stats = transform_nba_stats(raw_nba_stats)
    time.sleep(20)
    
    # If there are no games on the given day, unlike the other sports the MLB run will not fail quietly --> set up a try except with a visual warning
    try:
        raw_mlb_stats = get_daily_mlb_stats(day, month, year)
        mlb_stats = transform_mlb_stats(raw_mlb_stats)
        return {"NFL":nfl_stats, "NHL":nhl_stats, "NBA":nba_stats, "MLB":mlb_stats}
  
    except:
        print("An error was thrown pulling MLB stats. If the MLB is in season, investigate the error further.")
        time.sleep(6)
        mlb_stats = pd.DataFrame()
        clear_output(wait = False)

        
    return {"NFL":nfl_stats, "NHL":nhl_stats, "NBA":nba_stats, "MLB":mlb_stats}

In [68]:
# Build period stats for scoring
def assign_daily_stats(period, str_date, daily_data, rosters_file_path):
    str_date = str_date.split("/")[0].zfill(2) + "/" + str_date.split("/")[1].zfill(2) + "/" + str_date.split("/")[2].zfill(2)
    participant_rosters = get_current_rosters(rosters_file_path)
    
    # Pull the stats from the period so far is it's not the first day of the period, else start a new dict
    if str_date not in start_dates:
        current_period_stats = pkl.load(open("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Matchups Data/Period {}.pkl".format(period), "rb"))
    else:
        current_period_stats = {}
        for participant in waiver_participants:
            current_period_stats[participant] = {}
            for league in ["MLB", "NFL", "NHL", "NBA"]:
                current_period_stats[participant][league] = pd.DataFrame()
                
    # Check that we are in a 2 day transaction period, and if so, just return the current period stats unadjusted, in case this is run during that period
    # if str_date not in playing_dates:
    #     return current_period_stats

    for participant in waiver_participants:
        participant_starters = participant_rosters[participant]["Starters"]

        for league in ["MLB", "NFL", "NHL", "NBA"]:      
            if len(daily_data[league]) > 0:
                
                daily_league_data = daily_data[league]
                league_team_ids = participant_starters[participant_starters.League == league].ID
                team_daily_league_stats = daily_data[league]["players"][daily_league_data["players"]["Player ID"].isin([x for x in league_team_ids]) == True]
                if league == "NFL":
                    team_daily_league_stats = team_daily_league_stats.append(daily_data[league]["teams"][daily_league_data["teams"]["Player ID"].isin([str(x) for x in league_team_ids]) == True])
                # Add the data for the participants players for the day into the stats for the whole period so far
                if len(current_period_stats[participant][league]) != 0:
                    current_period_stats[participant][league] = current_period_stats[participant][league].append(team_daily_league_stats)
                else:
                    current_period_stats[participant][league] = team_daily_league_stats
                    
                # Attatch the position the players are starting in in the current period stats for later reference when scoring
                current_period_stats[participant][league]["Position"] = current_period_stats[participant][league]["Player ID"].apply(lambda x: participant_starters[participant_starters.ID == x].Position.iloc[0])
    
    
    pkl.dump(current_period_stats, open("/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Matchups Data/Period {}.pkl".format(period), "wb"))
                    
    return current_period_stats

In [69]:
# Now that we have the cumulative stats for all participants for a period, determine the matchups and score everything
def score_matchups(period, period_data, matchup_data_path):
    participants = pd.read_excel(matchup_data_path, sheet_name = "League View").iloc[[2,18]][["Unnamed: 2", "Unnamed: 7","Unnamed: 12", "Unnamed: 17"]]
    matchup_dict = {1:{participants["Unnamed: 2"].iloc[0]:{}, participants["Unnamed: 7"].iloc[0]:{}},
                    2:{participants["Unnamed: 12"].iloc[0]:{}, participants["Unnamed: 17"].iloc[0]:{}},
                    3:{participants["Unnamed: 2"].iloc[1]:{}, participants["Unnamed: 7"].iloc[1]:{}},
                    4:{participants["Unnamed: 12"].iloc[1]:{}, participants["Unnamed: 17"].iloc[1]:{}}}
    

    # Assign the stats for the participants team for the period to the participant
    for matchup in matchup_dict:
        for participant in matchup_dict[matchup]:
            for league in ["NFL", "MLB", "NHL", "NBA"]:
                matchup_dict[matchup][participant][league] = pd.DataFrame()
                if league in period_data[participant].keys():
                    df = period_data[participant][league]
                    if len(df) > 0:
                        matchup_dict[matchup][participant][league] = matchup_dict[matchup][participant][league].append(df)
      
    # Score the stats for the participant for the period
    for matchup in matchup_dict:
        for participant in matchup_dict[matchup]:
            for league in ["NFL", "MLB", "NHL", "NBA"]:
                if len(matchup_dict[matchup][participant][league]) > 0:
                    matchup_dict[matchup][participant][league]["Fantasy Points"] = matchup_dict[matchup][participant][league].apply(lambda x: assign_fantasy_points(x, league, scoring_dict, positional_scoring_dict), axis = 1)

    # Consolidate the matchup_dict
    scored_matchup_dict = {}
    for matchup in matchup_dict:
        scored_matchup_dict[matchup] = {}
        for participant in matchup_dict[matchup]:
            scored_matchup_dict[matchup][participant] = {}
            participant_period_stats = pd.DataFrame()
            for league in matchup_dict[matchup][participant]:
                df = matchup_dict[matchup][participant][league]
                if len(df) > 0:
                    participant_period_stats = participant_period_stats.append(df)
                    total_participant_period_points = participant_period_stats["Fantasy Points"].sum()
            
                scored_matchup_dict[matchup][participant] = {}
                starter_cols = ["Name", "Player ID", "Date", "Position", "Fantasy Points"]
                scored_matchup_dict[matchup][participant]["stats"] = participant_period_stats[starter_cols + [col for col in participant_period_stats.columns if col not in starter_cols]] if len(participant_period_stats) > 0 else pd.DataFrame()
                scored_matchup_dict[matchup][participant]["points"] = total_participant_period_points if len(participant_period_stats) > 0 else 0
    
    pkl.dump(scored_matchup_dict, open("All Sports Fantasy/Scored Matchups/Period {}".format(period), "wb"))
    
    return scored_matchup_dict

In [143]:
def output_league(period, scored_matchup_dict, year, month, day):
    # Write the League
    writer = pd.ExcelWriter('/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Matchup Printouts/Period {} - {}:{}:{}.xlsx'.format(period, year, str(month).zfill(2), str(day).zfill(2)), engine='xlsxwriter')
    participants = []
    v_indent_counter_I = 2
    v_indent_counter_II = 3
    
    
    for matchup in scored_matchup_dict:
        lens = []
        p1 = list(scored_matchup_dict[matchup].keys())[0]
        p2 = list(scored_matchup_dict[matchup].keys())[1]
        h_indent_counter = 3
        v_indent_counter = 2
        h_indent_counter_I = 1
        

        for participant in scored_matchup_dict[matchup]:
            df = scored_matchup_dict[matchup][participant]["stats"]
            p_cum = scored_matchup_dict[matchup][participant]["stats"].groupby(by = "Name", dropna = False).sum()
            lens.append(len(p_cum))
            df.to_excel(writer, sheet_name="{} vs. {}".format(p1, p2), startcol=h_indent_counter, startrow=v_indent_counter, index = False) # Default position, cell A1.
            p_cum.to_excel(writer, sheet_name="Cumulative Period Stats", startcol=h_indent_counter_I, startrow=v_indent_counter_I, index = True)
            
            
            v_indent_counter += len(df) + 3
            h_indent_counter_I += len(p_cum.columns) + 2
            
        v_indent_counter_I += max(lens) + 4    
        h_indent_counter_I += 2
        

    writer.save() 
    
    # Format the League
    
    workbook = openpyxl.load_workbook('/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Matchup Printouts/Period {} - {}:{}:{}.xlsx'.format(period, year, str(month).zfill(2), str(day).zfill(2)))
    
    standard_side = Side(border_style="thin", color = "000000")
    border = Border(top=standard_side, bottom=standard_side, right=standard_side, left=standard_side)
    alignment = Alignment(horizontal='center', wrap_text=False)
    gray_fill = PatternFill(start_color='B2B0B4', end_color='B2B0B4', fill_type='solid')
    orange_fill = PatternFill(start_color='F3A321', end_color='F3A321', fill_type='solid')
    bold = Font(bold=True)
    
    for matchup in scored_matchup_dict:
        p1 = list(scored_matchup_dict[matchup].keys())[0]
        p2 = list(scored_matchup_dict[matchup].keys())[1]
        worksheet = workbook["{} vs. {}".format(p1, p2)]
        
        df1 = scored_matchup_dict[matchup][p1]["stats"]
        df1_rows = len(df1)
        df1_columns = len(df1.columns)
        
        df2 = scored_matchup_dict[matchup][p2]["stats"]
        df2_rows = len(df2)
        df2_columns = len(df2.columns)
        
        df1_points = scored_matchup_dict[matchup][p1]["points"] if scored_matchup_dict[matchup][p1]["points"] != pd.DataFrame else 0
        df2_points = scored_matchup_dict[matchup][p2]["points"] if scored_matchup_dict[matchup][p2]["points"] != pd.DataFrame else 0
       
        
        # Alter the top df
        for col in [xlsx_columns[x+4] for x in range(df1_columns)]:
            for row in range(4, df1_rows+4):
                worksheet["{}{}".format(col,row)].border = border
                worksheet["{}{}".format(col,row)].alignment = alignment
            for row in range(3,4):
                worksheet["{}{}".format(col,row)].fill = gray_fill
        
            #worksheet.column_dimensions[col].bestFit = True
                    
        # Alter the bottom df         
        for col in [xlsx_columns[x+4] for x in range(df2_columns)]:
            for row in range(df1_rows + 7, df1_rows+7+df2_rows):
                worksheet["{}{}".format(col,row)].border = border
                worksheet["{}{}".format(col,row)].alignment = alignment
            for row in range(df1_rows + 6,df1_rows + 7):
                worksheet["{}{}".format(col,row)].fill = gray_fill
                
            worksheet.column_dimensions[col].bestFit = True
            
        # Put in scores in the matchups
        for col in [xlsx_columns[8]]:
            for row in [2, df1_rows + 5]:
                if row == 2:
                    worksheet["{}{}".format(col,row)].value = scored_matchup_dict[matchup][p1]["points"]
                    
                elif row == df1_rows + 5:
                    worksheet["{}{}".format(col,row)].value = scored_matchup_dict[matchup][p2]["points"]
                    
                worksheet["{}{}".format(col,row)].fill = orange_fill
                worksheet["{}{}".format(col,row)].alignment = alignment
                worksheet["{}{}".format(col,row)].border = border
                worksheet["{}{}".format(col,row)].font = bold
                
                
        # Format the Cumulative Stats
        worksheet = workbook["Cumulative Period Stats"]
        for col in [xlsx_columns[x+2] for x in range(df1_columns-3)]:
            for row in range(v_indent_counter_II, v_indent_counter_II + 1):
                worksheet["{}{}".format(col,row)].alignment = alignment
                worksheet["{}{}".format(col,row)].fill = gray_fill
                
        for col in [xlsx_columns[x+df1_columns] for x in range(df2_columns - 3)]:
            for row in range(v_indent_counter_II, v_indent_counter_II + 1):
                worksheet["{}{}".format(col,row)].alignment = alignment
                worksheet["{}{}".format(col,row)].fill = gray_fill
                
        v_indent_counter_II += max(lens) + 4
    
    
        
                
            
            
    
    workbook.save('/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Matchup Printouts/Period {} - {}:{}:{}.xlsx'.format(period, year, str(month).zfill(2), str(day).zfill(2)))

### Update The League (Run Above Functions) and Write to Excel

In [144]:
def run_league(year, month, day, start_dates):
    current_period = get_current_period(start_dates, year, month, day)
    
    roster_file_path = "/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Roster Sheets/Period {} Rosters.xlsx".format(current_period)
    
    daily_data = get_daily_league_stats(day, month, year)
    
    current_period_stats_by_team = assign_daily_stats(current_period, "{}/{}/{}".format(year, month, day), daily_data, roster_file_path)
    
    scored_matchup_dict = score_matchups(current_period, current_period_stats_by_team, roster_file_path)
    
    output_league(current_period, scored_matchup_dict, year, month, day)

In [145]:
roster_file_path = "/Users/jaredzirkes/Desktop/Python/All Sports Fantasy/Roster Sheets/Period 22 Rosters.xlsx"
y = assign_daily_stats(22, "2022/11/4", x, roster_file_path)
z = score_matchups(22, y, roster_file_path)
output_league(22, z, 2022, 11, 4)

In [91]:
# for n in range(25, 26):
#     print(n)
#     clear_output(wait=True)
#     run_league(2022, 10, n, start_dates)
#     time.sleep(10)